In [7]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import sentencepiece as spm
import spacy
import re

In [97]:
data = pd.read_csv("crimenewsdataset7.csv", sep=",")
data.head()
# Let's see how many categories we have here
print(f"Total unique categories are: {len(data['Category'].value_counts())}")
print(f"Count of occurance of each kEYWORD:")
data['Category'].value_counts()

Total unique categories are: 5
Count of occurance of each kEYWORD:


قتل وغارت        413
ظُلم و زیادتی    279
چوری             205
غیر متعلق         97
نشہ آور اشیاء     36
Name: Category, dtype: int64

In [48]:
spm.SentencePieceTrainer.Train('--input=crimenewsdataset4.csv --model_prefix=urdu_lm --vocab_size=1700')
sp = spm.SentencePieceProcessor()
sp.Load("urdu_lm.model")

True

In [3]:
mydata = pd.read_csv("expressnews.csv", sep=",")
mydata.head()
# Let's see how many categories we have here
# print(f"Total unique categories are: {len(mydata['Category'].value_counts())}")
# print(f"Count of occurance of each kEYWORD:")
# data['Category'].value_counts()

,Headline
0,ننکانہ صاحب میں بھائی نے 2 بہنوں کو قتل کردیا
1,سوتیلے بیٹے کے ہاتھوں ماں کے قتل میں سنسنی خیز...
2,لاہور میں پولیس اہلکار نے لین دین کے تنازع پر ...
3,لاہور میں پولیس کی کارروائی، ڈکیت میاں بیوی گر...
4,پتوکی میں ڈاکوؤں کی 15 سالہ لڑکی سے زیادتی، وز...


In [1]:
urdunews=[
    'نہاتی ہوئی خاتون کا ویڈیو بنا رہا تھا آئی آئی ٹی طالب علم ، موبائل میں ملے کئی فحش کلپ',
 'کنساس شوٹنگ کے بعد اب جنوبی کیرولینا میں ایک ہند نژاد تاجر کا قتل',
 'ایل جے پی لیڈر کے قتل کیس میں 4 لوگوں کے خلاف نامزد ایف آئی آر درج',
 'اداکارہ کنگنا رناوت کے ہیئر اسٹائلسٹ نے کیا انتہائی گھنونا کام ، ممبئی پولیس نے سیٹ سے ہی کیا گرفتار',
 'محبت سے شروع ہوا رشتہ شادی تک پہنچا ، مگر پھر کچھ ایسا ہوا کہ یکایک سب کچھ بدل گیا',
 'راکھی باندھنے کیلئے آئی نابالغ لڑکی کو دو دنوں تک یرغمال بناکر لڑکے نے کی آبروریزی ، کیس درج',
 'کٹھوعہ کیس: ملزمین کے وکیل کا متنازعہ بیان "خاتون ہے کتنا دماغ ہو گا اس میں"۔',
 'دہلی : آبروریزی کے مجرم اوبیر کیب کے ڈرائیور شیو کمار کو عمر قید کی سزا',
 'پونے : یتیم خانہ میں جنسی استحصال کے الزام میں مولوی گرفتار، آزاد کرائے گئے 36 بچے',
 'ٹیچر نے بھیجیں 13 سال کےاسٹوڈینٹ کو پورن ویڈیوز، ماں نے اس ایپ کے ذریعے کیا سنسنی خیز انکشاف',
 'گروگرام: دیوالی پر پہلے باپ کے ساتھ شراب پی، پھر اپنی ہی بہن کا کیا ریپ',
 'عاشق جوڑے کے ساتھ شرپسند عناصر نے کی چھیڑ چھاڑ ، لڑکی کا اتروایا کپڑا ، فحش ویڈیو بناکر کیا وائرل',
 'ساتویں کلاس کی بچی کی پانچ لوگوں نے کی عصمت دری، ویڈیو وائرل کرنے کی دے رہے تھے دھمکی',
 'بنگلورو کے سرکاری اسپتال کے آئی سی یو سے بچہ چوری',
 'ساتویں کی طالبہ کی اجتماعی آبروریزی کے بعد پڑوس کے لڑکوں نے فحش ویڈیو اور تصاویر کو کیا وائرل',
 'محبت کی شادی کے سبب لڑکی اورلڑکے کے ساتھ درندگی، کھمبے سے باندھ کر پٹائی، پھرپلایا پیشاب',
 'امریکہ میں رہائشی کمپلیکس میں فائرنگ، حملہ آور ہلاک',
 'پاکستانی فوج پر تنقید کرنے والی صحافی گل بخاری کی رہائی',
 'بنگلہ دیش میں صوفی عالم دین کا گولی مار کر قتل',
 'ممبئی میں شیوسینا لیڈر سچن ساونت کا گولی مار کر قتل',
 'نیم برہنہ حالت میں ملی لڑکی کی لاش، ریپ کے بعد کیا گیا نابالغہ کا قتل',
 'سسر نے بہو کے بیت الخلا میں لگوائے سی سی ٹی وی کیمرے، گندی نیت سے اس طرح اٹھا پردہ',
 'دو بھائیوں کا گولی مارکر قتل',
 'دہلی : گرل فرینڈ دوسرے لڑکی سے بڑھا رہی تھی دوستی ، بوائے فرینڈ نے کردی یہ وحشیانہ حرکت',
 'رمضان کے مہینہ میں خدا کی رضا حاصل کرنے کے لئے، والد نے کیا چار سالہ بیٹی کا قتل',
 'فحش ویڈیو بناکر شادی شدہ معشوقہ کو بلایا اپنے گھر اورپھر طمنچہ کی نوک پر دوستوں کے ساتھ مل کر کی اجتماعی آبروریزی',
 'تلنگانہ : انٹرکاسٹ لو میرج کی ملی سزا ، حاملہ بیوی کے سامنے ہی 23 سالہ نوجوان کا بے رحمی سے قتل',
 'نابالغ 5 سال کی بھتیجی کے ساتھ چچا نے کیا ریپ ، قتل کرکے جنگل میں پھینکا',
 'گریٹر نوئیڈا:اولا کیب میں خاتون کو یرغمال بناکر اجتماعی عصمت دری،حراست میں 6 مشتبہ',
 'دہلی: کتے سے چھوگیا آٹو، نشے کی حالت میں دوگروپ میں جھگڑا، ڈرائیورکا پیٹ پیٹ کرقتل',
 'میکسیکو میں منشیات کا دھندا کرنے والے گروہوں کی فائرنگ میں 30 افراد ہلاک',
 'آٹھ ماہ کی بچی کی لاش ملنے سے اندور میں سنسنی، عصمت دری کے بعد قتل کا خدشہ',
 'یکطرفہ محبت میں پاگل عاشق نے معشوقہ کے ساتھ کی یہ وحشیانہ حرکت ، پھر خود کو مارلی گولی',
 'نماز پڑھنے سے کیا انکار تو مامی اور بہن نے کر دیا نابالغ کا قتل',
 'معشوقہ نے دوپٹہ سے عاشق کو موت کے گھاٹ اتارا ، خودکشی کی شکل دینے کیلئے ہاتھ میں تھمایا لو لیٹر',
 'نابالغ لڑکی کی اجتماعی آبروریزی کی کوشش ، ویڈیو بناکر سوشل میڈیا پر کیا وائرل ، گرفتار',
 'جموں و کشمیر : اننت ناگ میں نامعلوم اسلحہ برداروں نے گولیاں برسا کر حریت لیڈر حفیظ اللہ میر کا کیا قتل',
 'شرمناک : باپ نے طمنچہ کی نوک پر 14 سالہ بیٹی کی آبروریزی کی ، کیس درج ، جانچ شروع',
 'قتل کے شک میں بھیڑ نے خاتون کو برہنہ گھمایا، آر جے ڈی لیڈر سمیت چھ گرفتار',
 'تھائی لینڈ میں بندوق بردار حملہ آوروں کی فائرنگ میں آٹھ افراد ہلاک',
 'راجدیو قتل معاملہ: نتیش کمار نے سی بی آئی جانچ کی سفارش کی',
 'گروگرام تشدد معاملہ میں ایک اور ملزم گرفتار، ہولی کے دن گھر میں گھس کر کیا تھا حملہ',
 'ہائی پروفائل سیکس ریکٹ کا پردہ فاش ، سوشل سائٹ کے ذریعہ پھیلا رکھا تھا جال ، پڑھیں کس طرح ہوتی تھی ڈیل',
 'مہاراشٹر: اورنگ آباد میں 60 سے 70 کے درمیان مدرسہ طالبات کی بگڑی حالت، ایک کی حالت نازک',
 'سات ماہ کی بچی کو ’بدقسمت‘ مان کرماں نے ڈوپٹے سے گھونٹ دیا گلا',
 'ریواڑی اجتماعی آبروریزی : اہم ملزمین میں سے ایک نشو پھوگاٹ گرفتار ، دو دیگر کی تلاش جاری',
 'اسکول میں 4 سال کی بچی کااس کے ہی ہم عمر لڑکے نے کیا جنسی استحصال، پرائیویٹ پارٹ میں درد ہونے پر ہوا انکشاف',
 'گریٹر نوئیڈا میں گئو اسمگلنگ کے شبہ میں گائے لے جا رہے دو لوگوں پر جان لیوا حملہ',
 'منگیتر کے ساتھ سیلفی لینا کالج کی طالبہ کو پڑگیا بھاری ، ناراض باپ نے پار کردی ساری حدیں',
 '! دوست کے خلاف ریپ کیس واپس لینا چاہتی تھی لڑکی تو پولیس افسر نے کیا ریپ',
 'اترپردیش : پانی پینے آئی خاتون کی عصمت دری کی کوشش ، ناکام ہونے پر پھینکا تیزاب',
 'دوستی کے نام پر نابالغ لڑکی کی دوستوں نے کی عصمت دری، دونوں ملزمین گرفتار',
 'کالج کی طالبہ سے کی لو میرج ، پھر دوستوں کے ساتھ مل کر کیا ریپ',
 'لو سیکس اور دھوکہ : ان کے بیڈ روم سے ہو رہے تھے کنڈوم غائب ، سچ سامنے آیا تو ... ۔',
 'ایک دن میں چار مرتبہ باپ کرتا تھا آبروریزی ، پھر بھی اس وجہ سے عدالت میں بولی لڑکی : آئی لو یو پاپا',
 'ہائی پروفائل جسم فروشی کے دھندے کا انکشاف، وہاٹس ایپ پر ہوتی تھی کسٹمرس سے ڈیل',
 'پیراگوئے میں فائرنگ میں چار افراد ہلاک، 11 زخمی',
 'اب دہلی کے ایک اسکول میں پانچ سالہ معصوم بچی کی آبروریزی ، ملزم گرفتار ، متاثرہ طالبہ کی حالت نازک',
 'ایک دن میں چار مرتبہ کرتا تھا باپ بیٹی کا ریپ، ایک بار کرایا اسقاط حمل ، دوسری مرتبہ حاملہ ہوئی تو کرایا یہ کام',
 'شاملی : گھر سے اغوا کرکے خاتون کی اجتماعی آبروریزی ، دو نوجوان گرفتار',
 'ہاپوڑ : اغو کے بعد بی اے کی طالبہ کی اجتماعی آبروریزی ، علاج کے دوران ہوئی موت',
 'بڑے سیکس ریکیٹ کا پردہ فاش ، کثیر تعداد میں قابل اعتراض اشیا برآمد، چار لڑکیوں سمیت چھ افراد گرفتار',
 'بوائے فرینڈ نے اپنے دوستوں کے ساتھ مل کر کیا معشوقہ کا ریپ ، متاثرہ کی حالت نازک',
 'منہ بولے پھوپھا نے 10 سال کی بچی کی کئی دنوں تک کی آبروریزی ، کچھ اس طرح ہوا انکشاف',
 'شرمناک: یہاں بھی چلا آن لائن سیکس ریکیٹ کا دھندہ ، اس طرح ہوتا ہے کام، اڑ جائیں گے ہوش',
 'بندوق لہرانے والے آشیش پانڈے نے کی خودسپردگی ، کہا : سیاسی لیڈر کا بیٹا ہوں ، اسلئے ہورہا میڈیا ٹرائل',
 'نابالغ بہن پر خراب ہوگئی تھی نیت ، سگے بھائی -بہن نے مل کردوسرے بھائی کے ساتھ کردی یہ وحشیانہ حرکت',
 'کیرالہ میں بھی دہلی کے نربھیا جیسا سانحہ، ریپ کے بعد طالبہ کا وحشیانہ قتل',
 'مدھیہ پردیش : مدد کرنے کے بہانے خاتون ریل مسافر کی اجتماعی آبروریزی',
 'شرمناک : اپنی ہی 11 سال کی بچی کا دو مہینے سے ریپ کر رہا تھا باپ',
 'اترپردیش میں 13 سالہ معصوم کی عصمت دری، ملزم پولیس کے حوالے',
 'کیلاش ستیارتھی کے گھر چوری، نقد-زیور کے ساتھ نوبل انعام کا توصیف نامہ بھی چرا لے گئے چور',
 'دوست کےساتھ بائیک پر جارہی تھی خاتون ، اغوا کرکے چار افراد نے کی اجتماعی آبروریزی ، ہائی وے پر چھوڑ کر فرار',
 'دہلی میں چار اور سات سال کی دو بچیوں کی عصمت دری، ایک نابالغ سمیت دو ملزمان گرفتار',
 'بزنس پارٹنر کا قتل کرلے لاش کے کئے 25 ٹکڑے ، سڑک پر پھینکے جسم کے اعضا',
 'ماں نے چار ماہ کے بچہ کی توڑ ڈالی 28 ہڈیاں ، رونے کی آواز سن کر باپ کو ملتا تھا سکون !۔',
 'صحافی پر 21 نابالغ لڑکوں کے جنسی استحصال کا لگا الزام ، کیس درج ، پولیس نے حراست میں لیا',
 'بڑے سیکس ریکٹ کا پردہ فاش ، ایک غیر ملکی سمیت 6 لڑکیاں پکڑی گئیں ، قابل اعتراض اشیا بھی برآمد',
 'رشتہ دار کی کرتوت ، دوستوں کے ساتھ مل کر کی آدیواسی خاتون کی اجتماعی آبروریزی ، حالت سنگین',
 'ٹک ٹاک پرلگائی جائے پابندی، مدراس ہائی کورٹ کی مدورائی بنچ نے مرکزکو دی ہدایت',
 'دوسری مرتبہ سیکس کرنے سے انکار کیا تو نوجوان نے اپنے پارٹنر کے ساتھ کردی یہ وحشیانہ حرکت',
 'اولا کیب ڈرائیور کو چار مسافروں نے بندوق کی نوک پر لوٹا ، بیوی کی بنائی نیوڈ کلپ',
 'میرٹھ میں چھیڑ چھاڑ سے پریشان طالبات ہوسٹل میں ہوئیں قید ، لگائے سنگین الزامات',
 'دودھ دینے آیا دودھ والا تو گھر میں خون سے لت پت ملی تین لاشیں ، دو معصوم زخمی حالت میں ملے',
 'لڑکی کی عصمت دری کے بعد اس کی ایم ایم ایس بنائی',
 'حیدرآباد میں سیکس ریکٹ کا پردہ فاش، فلم اداکارہ کو بچایا گیا',
 'ویلنٹائن ڈے پر نابالغ لڑکی کو بہلا پھسلاکر لے گیا سنسان جگہ ، پھر کی اجتماعی آبروریزی ، گرفتار',
 'ٹرائل روم میں کپڑے چینج کرنے کے دوران لڑکی کا ریپ اور ویڈیو بھی بنایا، اہل خانہ نے گھر سے نکالا',
 'نابالغ بچی نے دیا بچے کو جنم، سامنے آیا ریپ اورہوا یہ سنسنی خیز انکشاف، اڑ جائیں گے ہوش',
 'دہلی: انسانیت پھر ہوئی شرمسار، معصوم نابالغہ سے ’نربھیا‘ جیسی درندگی، ملزم گرفتار',
 'کٹھوعہ ریپ کیس: بچی سے گینگ ریپ کرنے کے لئے باہر سے بلائے گئے تھے لوگ',
 'نشیلاجوس پلاکراس کے ہی دوستوں نے لڑکی کو کیا بیہوش، پھر کار میں لے جاکر کیا یہ گندہ کام',
 'اسکول کی دوست نے وہاٹس ایپ پر سونوگرام بھیج کر لکھا "میں تمہارے بچہ کی ماں بننے والی ہوں "، اور پھر ...۔',
 'حاملہ اداکارہ کا قتل: پیٹ میں مارے اتنے چاقو کہ حمل میں پل رہا 8.5 ماہ کا بچہ فرش پر گرا',
 'جادوگر کی صلاح پر ماں ۔ باپ نے 6 سال کی بیٹی کو مارا اور پھر زمین میں دفنا دیا',
 'پنچکولہ میں سیکس ریکٹ کا پردہ فاش،4 خواتین سمیت 6 گرفتار',
 'سدھارتھ نگر میں 6 سالہ معصوم کے ساتھ ریپ ، بارات دیکھنے نکلی تھی بچی',
 'جعلی مارک شیٹ بنانے کے الزام میں تین سال کی قیدبامشقت',
 'پرنسپل اور ٹیچر 9 مہینے تک کرتے رہے 5 ویں کلاس کی طالبہ کا ریپ، لیتے تھے سیلفیاں، حاملہ ہونے پر ہوا انکشاف',
 'شوٹنگ کے بہانے اسٹوڈیو بلاکر ماڈل کا جنسی استحصال ، برہنہ ویڈیوز اور تصاویر نکال کر بدنام کرنے کی دھمکی',
 'جنسی تعلقات قائم کرنے سے انکار کیا تو خاتون نے چمٹے سے جلادیا 13 سال کے بچے کا پرائیویٹ پارٹ',
 'بہار کے جہان آباد میں سرعام لڑکی سے چھیڑچھاڑ کا ویڈیو وائرل، چار گرفتار',
 'جموں و کشمیر : کٹھوعہ واقعہ کے سال بھر بعد بکروال برادری کی ایک اور لڑکی کی آبروریزی ، کرایا گیا اسقاط حمل',
 'ٹیکساس میں فائرنگ، حملہ آور سمیت آٹھ افراد ہلاک',
 'یوگی راج میں جنگل راج: ہائی وے پر خاندان کو یرغمال بنا کر 4 خواتین سے گینگ ریپ',
 'بچے کو پنجرے میں قید کرکے بھوک سے مارنا چاہتا تھا باپ ، ماں بھی دے رہی تھی ساتھ ، وجہ جان کر اڑجائیں گے ہوش',
 'شرمناک : چوری کے الزام میں بھیڑ نے پہلے بچے کی پٹائی کی اور پھر ننگا کرکے پورے بازار میں گھمایا',
 'ہریانہ میں طالب علم نے پرنسپل کا گولی مارکر قتل کیا، پولیس نے کیا گرفتار',
 'یوگی راج میں خواتین اور بچیاں محفوظ نہیں، بریلی میں دو معصوم بچیوں کی عصمت دری کی وحشیانہ واردات',
 'نشیلی اشیا پلا کر طالبہ کی آبروریزی کرکے بنائی فحش ویڈیو ، پھر دھمکی دے کر کی فرضی شادی',
 'بیوی کا قابل اعتراض ویڈیو دیکھتے ہی آپا کھوبیٹھا شوہر، پھر کیا ایسا کام کہ جان کر اڑجائیں گے آپ کے ہوش',
 'نابالغ لڑکی کے ساتھ عصمت دری کے معاملے میں دو افراد گرفتار',
 'امریکہ میں برہنہ شخص کی فائرنگ سے چار افراد ہلاک، دو دیگر زخمی',
 'سرپنچ انتخابات میں ووٹ نہیں ڈالنے پر دبنگوں نے کی پورے خاندان کی جم کر پٹائی',
 'لفٹ دینے کے بہانے کار میں بیٹھایا ، پھر نشیلی چیز پلا کر 26 سالہ شادی شدہ خاتون کی اجتماعی آبروریزی',
 'میرٹھ میں دو سگے بھائیوں نے چار سالوں تک اپنی ہی بہن کی آبروریزی کی ، گرفتار',
 'قضائے حاحت کیلئے گھر سے باہر گئی تھی انٹر کی طالبہ ، گھات میں بیٹھے حیوانوں نے دبوچ لیا اور پھر کی یہ گھنونی حرکت',
 'جھارکھنڈ: اجتماعی عصمت دری کے بعد سابق شوہر نے خاتون کے پرائیویٹ پارٹ میں ڈالی لکڑی',
 'آبروریزی کے بعد نابالغ لڑکی کا بے رحمی سے قتل ، سر کاٹ کر اپنے ساتھ لے گئے مجرمین',
 'گرل فرینڈ کا ہوٹل میں کیا ریپ، پھرکیا ایسا کام کہ ہوٹل اسٹاف اور پولیس بھی حیران',
 'شرمناک: پاکستان میں قرض کی رقم واپس نہ کرپانے پر لڑکی کی ’عصمت دری',
 'اترپردیش : تین دنوں تک کلینک میں بند کرکے 13 سالہ بچی کی آبروریزی کرتا رہا ڈاکٹر ! ، گرفتار',
 'حیدر آباد : شراب کے نشے میں حیوان بنا باپ ۔ 3 سال کے معصوم کو بے رحمی سے پیٹا',
 'گریٹر نوئیڈا میں ماں بیٹی کا سفاکانہ قتل، معاملہ میں ایف آئی آر درج، تحقیقات شروع',
 'آبروریزی کے الزام میں بی جے پی لیڈر حکم سنگھ گرفتار، پہلے بھی جا چکا ہے جیل',
 'چھ سالہ بچی کو اغوا کرکے عصمت دری، سنگین حالت میں اسپتال میں بھرتی',
 'بڑے سیکس ریکٹ کا پردہ فاش، قابل اعتراض حالت میں تین لڑکیاں اور 4 لڑکے گرفتار',
 'لڑکی نے 35 مرتبہ ٹھکرایا تھا پرپوزل ، سرپھرے عاشق نے چاقو سے 35 وار کرکے لے لی جان',
 'ترکی: 85 سالہ شوہر رہتا تھا سوشل میڈیا پر سرگرم، بیوی کو ہوئی جلن تو کر دیا قتل',
 'بیوی موٹی ہو گئی تو شوہر نے کہہ دیا طلاق ، طلاق ، طلاق ، پولیس نے کیا گرفتار',
 'اوبر کیب ڈرائیور نے خاتون سے کی گندی بات ، 100 ڈالر کے بدلے سیکس کا دیا آفر ',
 'پاکستان میں ہندو ڈاکٹر کا گولی مار کر قتل',
 'ماں نے ناجائز تعلقات کا بھانڈا پھوٹ پڑنے کے بعد سگے جوان بیٹے کو زہر دے کر قتل کردیا',
 'آدھی رات کو دروازہ پیٹنے لگا کانگریسی لیڈر ، عزت بچانے کیلئے طالبہ نے مچادیا شور اور پھر ہوا کچھ ایسا ...۔',
 'کانپور میں 96 کروڑ روپے کے پرانے نوٹ برآمد، 16 افراد حراست میں',
 'ونیتا نندا کے الزام پرآلوک ناتھ کا جواب "میں نے نہیں کسی اورنے کی ہوگی آبروریزی"۔',
 'چھتیس گڑھ: نوجوان کے قتل معاملہ میں ماں باپ سمیت تین گرفتار',
 'پیار کے جال میں پھنسا کر عاشق ایک سال تک کرتا رہا آبروریزی ، شادی کی بات پر اب دے رہا ہے یہ دھمکی',
 'کرناٹک : پستول فروخت کرنے کی کوشش کرتے ہوئے کنڑ اداکار سمیت چارگرفتار',
 'تین لڑکوں نے نابالغ لڑکی کا اغوا کرکے اجتماعی عصمت دری کی',
 'سیکس پاور کی دوائیں کھاکرشوہرکرتا تھا ریپ ، برداشت کی حد ٹوٹ گئی تو بیوی نے اٹھایا یہ قدم',
 'ساتویں جماعت کی طالبہ کی مندر میں آبروریزی ، سنگین حالت میں اسپتال میں داخل',
 'باپ نے کی اپنی ہی 13 سال کی بیٹی کی آ بروریزی ، ملزم والد گرفتار',
 'اترپردیش : 22 سال کے نوجوان پر لگا 70 سال کی خاتون کی آبروریزی کا الزام ، سی سی ٹی وی میں ہوا قید',
 'یکطرفہ پیار کے معاملہ میں پنچایت نے سنایا حیران کن فیصلہ ، لڑکی کا بھائی پہنچا گیا تھانہ',
 'نابالغ دوشیزہ کی اجتماعی آبروریزی، پولیس نے تینوں ملزمان کو کیا گرفتار',
 'اپنی نوعیت کا پہلا کیس ، لڑکیوں سے چھیڑ چھاڑ کے معاملے میں ایک لڑکی کو ملی تین سال کی سزا',
 'الور: 7 ماہ کی بچی سے ریپ کرنے والے کو سزائے موت ، نا بینا تائی کی گود سے چھین کر لے گیا تھا معصوم کو',
 'بیوی کے ساتھ جنسی تعلقات بناتے وقت پورن فلم چلاکر لیپ ٹاپ کا ویب کیمرا آن رکھتا تھا شوہر',
 'کانپور: لڑکی کو ملنے بلایا اور پھر تین دوستوں نے کی اجتماعی عصمت دری',
 'بہار: اطفال تربیت گاہ میں ہوتا تھا بچوں کا جنسی استحصال ، چیئرمین گرفتار',
 'نا بالغ والی بال کھلاڑی نے کوچ پر لگائے الزام ، ہوٹلوں میں لے جا کر کرتا تھا ریپ',
 'دلت طالبہ کی اجتماعی عصمت دری : ملزمین کے خلاف سخت کارروائی کا خواتین تنظیموں نے کیا مطالبہ',
 'دہلی سے شاملی کو جانے والی پیسنجر ٹرین میں عالم دین کے قتل کی کوشش',
 'امریکہ کی پہلی مسلم خاتون جج شیلا عبدالسلام کی لاش ہڈسن ندی سے برآمد',
 'اترپردیش : رام پور میں چھ سال کی بچی کی عصمت دری کی کوشش کی پاداش میں داروغہ معطل',
 'دو عاشقوں کے ساتھ مل کر اس خاتون نے کردیا نابالغ بہن کاقتل، وجہ جان کر اڑجائیں گے ہوش',
 'نوئیڈا کے جی آئی پی مال کے پاس دن دہاڑے 20 لاکھ کی لوٹ',
 'جسم فروشی معاملے میں چار افراد گرفتار، پوچھ گچھ جاری',
 'دہلی کے ایک پرائیویٹ اسکول میں 4 سال کی بچی سے چھیڑ چھاڑ، مقدمہ درج',
 'سیکس ٹوائے کا استعمال کرکے 19 سالہ لڑکی نے سہیلی کی آبروریزی کی ، کیس درج ، گرفتار',
 'گیا میں جیتن مانجھی کے قافلے پر حملہ، گاڑی پھونکی، جان بچا کر بھاگے',
 'خاتون کا جنسی استحصال کرنے کے الزام میں آر ایس ایس کا مبینہ پرچارک دلی سے گرفتار',
 'عضو تناسل کاٹ کر فٹ بالر کے قتل کی وجہ کیا تھی، ریپ یا سیکس تعلق ؟',
 'اغوا کے بعد تین سال کی معصوم کی آبروریزی ، قبرستان کے نزدیک خون سے لت پت ملی ، حالت نازک',
 'اترپردیش : بستی میں سنت کٹیر آشرم کے چار باباوں پر اجتماعی آبروریزی کا الزام ، مقدمہ درج ، جانچ شروع',
 'ماں مجھے معاف کر دینا، میں اب اور نہیں جی سکتی، میری عزت برباد ہو گئی',
 'روہتک کے نالے میں ملی 6 سال کی بچی کی لاش، ریپ کے بعد قتل کا اندیشہ',
 'عشق ومحبت کی وجہ سے ایک شخص نے بھائی کے ساتھ مل کر بیوی کا کردیا قتل',
 'اترپردیش : بریلی میں 14 سال کی طالبہ کی اجتماعی آبروریزی ، پھر پنچایت نے کروادی بچہ شادی !۔',
 'شرمناک : اسکول میں پرنسپل نے طالبہ سے کی چھیڑ چھاڑ ، جم کر ہنگامہ ، معاملہ درج',
 'اسرائیل میں فائرنگ ، ایک شخص کی موت، آٹھ زخمی',
 'آئی پی ایل کھلاڑی کے خلاف 27 سالہ لڑکی کو اغوا اورچھیڑ چھاڑ کرنے کا کیس درج',
 'عشق کی راہ میں آڑے آیا مذہب، بالآخرعشق کا ہوا قتل',
 'پاکستان میں مسلکی تشدد میں پانچ افراد ہلاک',
 'لڑکیوں کو گندے میسیج اور فحش فوٹو بھیجنے والا نکلا داروغہ کا بیٹا ، گرفتار',
 'ٹی وی اداکارہ روپا گانگولی پر حملہ، ملزمان کو پولیس نے کیا گرفتار',
 'سماج وادی پارٹی کے دفتر میں خاتون لیڈر سے مارپیٹ، ہنگامہ',
 'گروگرام : آبروریزی کے بعد تین سال کی بچی کا بے رحمی سے قتل ، پرائیویٹ پارٹ میں ڈالی لکڑی',
 'نا بالغ سے آٹو ڈارائیور نے کی عصمت دری ، پولیس نے دو کو کیا گرفتار',
 'اترپردیش : سماجوادی پارٹی لیڈر سمیت تین افراد پر اسکول میں یرغمال بناکر لڑکی کی آبروریزی کا الزام',
 'فتح پور: کوچنگ میں پڑھنے جا رہی نابالغہ کو اغوا کر پڑوسی نے کی عصمت دری',
 'شرمناک : بھینس کے بچوں کے ساتھ ریپ ، قصوروار کو ملی 6 سال کی سزا',
 'کینیڈا میں قتل کے بعد خود کشی معاملے، چار ہلاک',
 'خاتون کی اجتماعی آبروریزی کے چار ملزموں کو 20 ، 20 سال قید کی سزا',
 'خاتون کو فروخت کرکے جسم فروشی کے دھندہ میں دھکیلا ، 10 دنوں تک کی جاتی رہی لگاتار اجتماعی آبروریزی',
 'ڈی ایس پی نے کی نابالغ طالبہ کے ساتھ فحش حرکت ، کیس درج ، گرفتار',
 'امریکی یونیورسٹی میں 11 لوگوں کو زخمی کرنے والا کون تھا؟',
 'آن لائن سیکس ریکیٹ کا پردہ فاش ، روسی لڑکی سمیت چار گرفتار',
 'شادی کے 3 دن بعد دلہن کا اغوا ، بندوق کی نوک پر پہلی بیوی نے کرائی اجتماعی عصمت دری',
 'چرخی دادری: گھر جا رہے باپ بیٹے کی کاٹی داڑھی، جان سے مارنے کی دی دھمکی',
 'وڈودرا میں جرمن لڑکی کی عصمت دری، متاثرہ نے کہا واقعہ کے بعد تین دنوں تک سو نہیں پائی',
 '...لو سیکس اور دھوکہ : اپنے ہی داماد کو کال گرل کے ساتھ سسر نے دیکھا لائیو اور پھر ہوا یہ',
 'پاکستان کے حیدرآباد میں دھماکہ، 24 زخمی',
 'یوپی پولیس کا بلند شہر میں گینگ ریپ کے شرمناک واقعہ کے کلیدی ملزم کی شناخت کا دعویٰ',
 'خواتین کا قتل کر کے اس کے اعضا نکال لیتا تھا یہ سیریل کلر ، اب 130 سال بعد ہوئی شناخت',
 "گوا کی خاتون کانگریس سکریٹری کا الزام، ' بی جے پی لیڈر کے حامی نے دی گینگ ریپ کی دھمکی'۔",
 "دھمکی دیکر 'سیکس ریکٹ' چلا رہی تھی یہ اداکارہ، خواتین کورکھا جاتا تھا بھوکا",
 'باپ ہی بن گیا حیوان، 20 لوگوں کے ساتھ مل کراپنی ہی بیٹی کی کرتا رہا آبروریزی',
 'باپ سے بدلہ لینے کے لئے بیٹی کی اجتماعی آبروریزی ، ملزم باپ اور بیٹا گرفتار',
 'گھوڑے کو بیہوش کرنے والے انجیکشن 6 سال کی بچی کو لگاکر 3 مرتبہ رپپ کے بعد قتل، تا عمرقید کی سزا',
 'انتہائی شرمناک ! ٹیوشن سے لوٹ رہی آبروریزی کی متاثرہ لڑکی کو زبردستی پلایا گیا زہر ، اسپتال میں داخل',
 'اسلامک اسٹیٹ نے موصل کے قریب قتل عام کیا',
 'اب یوپی کے ایٹہ میں ’ کٹھوعہ واقعہ‘، عصمت دری کے بعد 8 سالہ بچی کا قتل',
 'اترپردیش : نوجوان پر بکری کے ریپ کا لگا الزام ، پولیس جانچ میں مصروف',
 'دہلی: لڑکی کی پٹائی کا ویڈیو وائرل، وزیرداخلہ نے دیئے سخت کارروائی کے احکامات',
 'کڑکڑڈوما کورٹ میں مارے گئے کانسٹیبل کے گھر والوں کو ایک کروڑ کا معاوضہ',
 'بلند شہر : حلالہ کے خلاف آواز اٹھانے والی تین طلاق متاثرہ مسلم خاتون پر تیزاب حملہ ، حالت سنگین',
 'اب راکی کی ماں منورما دیوی فرار، پولیس گرفتاری کی تیاری میں، منورما دیوی پارٹی سے معطل',
 'نشیلی اشیا پلاکرسی بی ایس ای ٹاپر19 سالہ طالبہ کے ساتھ 12 لڑکوں نے کی اجتماعی آبروریزی، معاملہ درج',
 'یوگی راج میں ہندو یوا واہنی کی غنڈہ گردی: مسلم بزرگ کو پیٹ پیٹ کر مار ڈالا',
 'عاشق کے ساتھ بستر پر بیٹھی تھی حاملہ ماں ، چار سال کے بیٹے نے کیا ایسا کام کہ جان کر اڑ جائیں گے ہوش',
 'فلوریڈا نائٹ کلب شوٹنگ: کم از کم دو افراد ہلاک، 16 دیگر زخمی',
 'نوح : 85 سالہ سن رسیدہ شخص نے 6 سال کی معصوم بچی کی آبروریزی کی ، پولیس نے کیا گرفتار',
 'داعش نے 40 شہریوں کی لاشیں بجلی کے کھمبے سے لٹکائیں',
 'آرمی جوان نے بنایا نابالغ لڑکے ۔ لڑکی کا برہنہ ویڈیو ، کی چھیڑ چھاڑ ',
 'مسلح افراد نے قندھار میں افغان ہوائی اڈے کے عملہ کی پانچ خواتین کو ہلاک کردیا',
 'سہاگ رات میں رومانس کے موڈ میں تھا شوہر ، مگر بیوی نے فون کر کے اپنے عاشق کو بلالیا ، پھر وہ ہوا کہ اڑ گئے سب کے ہوش',
 'ایسے شروع ہوا تھا یہ پیار ، لڑکی نے سیکس سے منع کیا تو کردی یہ وحشیانہ حرکت',
 'لکھنو میں انسانیت شرمسار ، کینسر متاثرہ نابالغ لڑکی کی اجتماعی آبروریزی ، مدد مانگنے پر راہگیر نے بھی کیا ریپ',
 'اترپردیش میں ایک شخص نے بیوی اور 3 بیٹیوں کوزندہ جلایا',
 'آئی اے ایس افسر کی بیٹی کا پیچھا کرنے کے الزام میں ہریانہ بی جے پی صدر کا بیٹا گرفتار',
 'بیس سال کی لڑکی کا 4 سال تک کیا ریپ، حاملہ ہونے پر شخص نے کیا یہ',
 'نابالغ 4 سالہ معصوم کی عصمت دری کرنے والے ٹیچر کا معاملہ ہائی کورٹ پہنچا',
 'جام کا بہانہ بناکر ہائی وے سے سنسان جگہ پر لے گیا آٹو ڈرائیور ، طمنچہ کی نوک پر کی لڑکی کی آبروریزی',
 'راجستھان میں معمر شخص کی خودسوزی',
 'شرمناک: ریپ کے بعد کیا گیا تھا قتل، 11 سال کی بچی کی اس حالت میں ملی لاش کہ جان کر کانپ اٹھیں گے آپ',
 'تھانے میں 16 کروڑ کی سنسنی خیز لوٹ، 8 لٹیروں نے لوٹ کی اس واردات کو دیا انجام',
 'ابو عاصم اعظمی کو جان سے مارنے کی دھمکی، مقامی قلابہ پولیس تھانے میں کی شکایت',
 'اترپردیش : بلند شہر میں مشتبہ حالات میں دو بہنوں کی جلی لاشیں برآمد ، ابتدائی جانچ میں گلا دبا کر قتل کا شبہ',
 'جموں و کشمیر : گھر لوٹ رہی خاتون کو گھسیٹ کر لے گیا جنگل ، آبروریزی کی ، بتانے پر سنگین نتائج کی دی دھمکی',
 'طالبات سے بد کاری کرانے والے گروہ کا پردہ فاش ، وہاٹس ایپ سے ہوتی تھی لڑکیوں کی بکنگ',
 'الہ آباد میں بی ایس پی لیڈر راجیش یادو کا گولی مارکر قتل، کارکنان نے بس کو کیا نذر آتش',
 'چھیڑ چھاڑ سے بچنے کے لئے تین لڑکیاں چلتی بس سے كودیں',
 'یوایس بیوٹی کنٹیسٹ کی فاتح نے نابالغ طالب علم کو بھیجی نیوڈ فوٹوز ، جنسی ہراسانی کا کیس درج',
 'انسانیت شرمسار ، بلند شہر میں 7 اور 5 سال کی دو معصوم بہنوں کی آبروریزی',
 'سیکس کریمنل کے ساتھ ڈیٹنگ کررہی تھی ایتھیلیٹ ، بریک اپ کیا تو چکانی پڑ گئی یہ بڑی قیمت',
 'ویڈیو: مدھیہ پردیش میں دبنگئی، خاتون کے ہاتھ اور پیر کاٹے گئے',
 'امریکی یونیورسٹی کے قریب فائرنگ میں 2 افراد ہلاک',
 'میکسیکو میں فائرنگ کے تبادلے میں کم از کم پانچ افراد ہلاک',
 'دادری سانحہ: مشتعل بھیڑ اخلاق کے پورے خاندان کو زندہ جلا دینا چاہتی تھی',
 'چاکلیٹ کیلئے کانسٹیبل کرتا تھا خاتون کو بلیک میل ، ہوٹل میں لے جاکر کی آبروریزی ، گرفتار',
 'بریلی : بیٹے کی چاہت میں بے رحم باپ نے معصوم بیٹی کو چھت سے نیچے پھینکا ، حالت نازک',
 'نرسنگ ہوم میں یرغمال بناکر ڈاکٹر نے نابالغ لڑکی کی آبروریزی کی ، کیس درج',
 'دربھنگہ قتل کیس : ایس پی نے تھانہ انچارج کو معطل کیا ، تحقیقات کے لیے پہنچی ایس آئی ٹی کی ٹیم',
 'دوشیزہ کی لاش برآمد ، جس کی آبروریزی کے بعد مجرموں نے کر دیا قتل',
 'لڑکیوں کے جسم پر ملے جلنے۔ کٹنے کے نشان، عصمت دری سے پہلے لگایا جاتا تھا نشہ کا انجکشن',
 'اس 38 سال کے شخص نے 11 سال کی بچی کو بنایا بیوی ، پھر سامنے آئی دل دہلانے والی کہانی',
 'پونے : ایک سالہ بچی سے عصمت دری کے بعد کیا قتل، ملزم گرفتار',
 'اجمیر میں نابالغ کی اجتماعی آبروریزی کی کوشش، دیہاتیوں نے 8 گھنٹے تک درخت پر لٹکا کر دی سزا',
 'لو سیکس اور دھوکہ: شوہر کو اس حالت میں پکڑا جہاں ثبوت کی ضرورت نہیں ہوتی',
 'گولیوں کی تڑتڑاہٹ سے گونج اٹھا ہریانہ کا سرکاری اسکول، ایک ٹیچر کی موت',
 'بنگلورو:چلتی گاڑی میں 23 سالہ لڑکی کی عصمت دری ، ملزمین کو 14دنوں کی پولیس تحویل بھیجا گیا',
 'انسٹا گرام پر گندی تصویریں کیوں ڈالیں؟ " 50 طلبا کی مصیبت بنی ایک لڑکی کی مستی "',
 'دہلی کی ماڈل ویشالی راجوريا بنی جرائم کی ملکہ، لوگوں کو ایسے کر دیتی تھی کنگال',
 'نابالغ بھتیجی کا چاچا نے کیا ریپ، بیمار ہونے پر بچی نے ڈاکٹر کو بتائی آپ بیتی',
 'شادی کے تیسرے دن سوال کھڑا ہوا کہ دلہن کا شوہر ہے کون ؟',
 'یٹہ میں عصمت دری کے بعد نو سال کی بچی کا قتل',
 'پیار کے جال میں پھنسا کر لڑکی سے اینٹھے 21 لاکھ روپے ، ہوٹل میں لے جا کر کی آبروریزی',
 'کتیا کے ساتھ ریپ کی کوشش ، پرائیویٹ پارٹ میں ڈالی لوہے کی راڈ',
 'سان فرانسسکو میں گودام میں ہوئی فائرنگ میں چار افراد ہلاک',
 'پہلے ریپ کیا پھر بنایا ویڈیو، متاثرہ نے کورٹ میں درج کرایا بیان',
 'میلہ گھومنے کیلئے گھر سے نکلا تھا فوجی جوان ، بندوق کی نوک پر رشتہ داروں نے کرادی شادی',
 'بہار میں آرٹی آئی کارکن سمیت دولوگوں کا گولی مارکر قتل، کیس درج',
 'وائف سویپنگ کے سنسنی خیز معاملہ کا انکشاف ، فحش ویڈیو بناکر دوستوں کو خوش کرنے کیلئے بلیک میل کرتا تھا شوہر  ',
 'بھاگل پور : جاگرن پروگرام کے دوران فائرنگ میں نوجوان ہلاک اور دو زخمی',
 'اردن میں اسرائیلی سفارت خانے میں فائرنگ کے تبادلے میں دو افراد کی موت',
 'دہلی یونیورسٹی کی طالبہ کے ساتھ چلتی بس میں چھیڑ چھاڑ، کیس درج ، مدد مانگنے پر بھی کوئی نہیں آیا سامنے',
 'بیوی نے کی خود کشی، لاش کو کار میں 8 گھنٹے تک لے کر گھومتا رہا شوہر',
 'غائب ہوئے موبائل میں تھی بوائے فرینڈ کے ساتھ بنائی فحش ویڈیو، خاتون ڈاکٹر سے دو لاکھ روپے کا مطالبہ',
 'کھانا میں نشیلی اشیا ملا کر کیا بے ہوش ، پھر عضو خاص کاٹ کر بنا ڈالا ہیجڑا',
 'باپ نے کی چارمہینہ تک اپنی بیٹی کے ساتھ عصمت دری، ملزم گرفتار',
 'نالندہ: نابالغہ کی لاش برآمد، اجتماعی عصمت دری کے بعد قتل کا شک',
 'عصمت دری متاثرہ کا اسقاط حمل کرنے کے سبب کانگریس کی خاتون لیڈر گرفتار',
 'پاکستان میں آٹھ سالہ بچی کا آبروریزی کے بعد قتل، ہزاروں افراد کا مظاہرہ',
 'بی جے پی کے سابق ممبر اسمبلی سمیت 15 لوگوں پر اجتماعی آبروریزی کا الزام ، ایف آئی آر درج',
 'حلالہ کے نام پر سسر نے کی آبروریزی ، دو مولویوں سمیت پانچ افراد کے خلاف کیس درج ، جانچ جاری',
 'دھوکے سے بنایا ایڈز کا مریض، تو کر دیا دوست کی ماں کا قتل',
 'گرل فرینڈ سے کال گرل بنی بالی ووڈ اداکارہ، جس کی عاشقی میں شوہر نے کیا بیوی کا قتل',
 'دبنگ باپ کے بیٹوں نے کلاس میں دکھائی دبنگئی، پہلے بیچ کے ساتھی کو پیٹا پھر ویڈیو کیا وائرل',
 'غیرت کے نام پر قتل سے قبل سامعہ کی عصمت دری کی گئی',
 "' میدان گڑھی سے 18 لڑکیاں آزاد: ' بیرون ممالک ان لڑکیوں کو بھیج کر ان سے کرائی جاتی جسم فروشی",
 'نہانے گئی نو سالہ بچی کی عصمت دری کے بعد قتل، ایک شخص گرفتار',
 'دلت نابالغ لڑکی کی اجتماعی عصمت دری، ایم ایم ایس بھی بنایا',
 'امریکہ میں برقع پوش مسلم خاتون کو جلا دینے کی دھمکی، پولیس معاملہ کی جانچ میں مصروف',
 'چچا نے اپنی نابالغ بھتیجی کو کلہاڑی مار کر بے رحمی سے قتل کر دیا',
 'دو بہنوں کا کیا اغوا ، اجتماعی آبروریزی کے بعد ایک کو کھیت میں چھوڑا ، دوسری لاپتہ',
 'شوہر نے اپنے سامنے ہی کروائی نابالغ بیوی کی عصمت دری، دونوں ملزم گرفتار',
 'بہلا پھسلا کر لے گیا گھر ، زبردستی مانگ میں بھرا سندور اور کی آبروریزی ، پھر سوشل میڈیا پر پوسٹ کردی قابل اعتراض تصویریں',
 'آسٹریلیا میں برطانوی خاتون کی عصمت دری',
 'سیکس سے انکار کرنے پر ممبئی میں 20 سالہ ماڈل کا کردیا گیا قتل',
 'جنسی تعلقات بناتے وقت دم گھٹنے سے ہوئی تھی گرل فرینڈ کی موت ، بوائے فرینڈ کے خلاف کیس درج',
 'راجیندر نگر میٹرو اسٹیشن میں لوٹ کی واردات، چاقو مار کر میٹرو ملازم سے لوٹ لئے بارہ لاکھ',
 'لڑکے نے گرل فرینڈ کی ایسی شرمناک تصویریں کی وائرل اور سیکس ویڈیو بھی پوسٹ کرنے کی دی دھمکی',
 'بھائی کو اعتماد میں لے کر بہن سے کیا عشق ، پھر معمولی بات پر عاشق نے کردی یہ وحشیانہ حرکت',
 'ڈی کمپنی کے تین کارندے بلند شہر سے گرفتار، کر رہے تھے بڑی منصوبہ بندی',
 'فوج کے جوان کا الزام، بیوی نے پرائیویٹ پارٹ پر زہریلا مرہم لگاکر رچی تھی یہ سازش',
 'ہماچل پردیش: غیرقانونی تعمیرات توڑنے گئی خاتون آفیسر کا گولی مار کر قتل',
 'لفٹ لینے سے کیا انکار ، تو گھر جا کر طالبہ کے چہرے پر پھینکا تیزاب',
 'لا پتہ بچیوں نے سنائی ریپ کی داستان، بولیں میرے ساتھ ہوا یہ غلط کام',
 'دہلی پولیس نے سیریل ریپسٹ کو کیا گرفتار، سینکڑوں اسکولی بچیوں کو بنایا شکار',
 'سولہ سال کی عمر میں اس ماڈل کی ہوئی تھی آبروریزی ، 32 سال بعد کیا انکشاف ، خاموشی کی بھی بتائی وجہ',
 'دارالحکومت پٹنہ میں لالو یادو کے قریبی آر جے ڈی لیڈر کا گولی مار کر قتل',
 'امریکہ میں فلوریڈا ہوائی اڈے پر اندھا دھند فائرنگ، پانچ افراد ہلاک، آٹھ زخمی',
 'پولیس اہلکار نے خاتون فیس بک دوست کو ہوٹل کے کمرے میں لے جاکر کیا ریپ ، بنایا فحش ویڈیو',
 'مشرقی دہلی کے کرشنا نگر علاقے میں کل رات فائرنگ میں چار افرادشدید زخمی',
 'پورن دیکھنے کے بعد 9 سے 14 سال کے لڑکوں نے کی 8 سالہ بچی کی عصمت دری',
 'چھ مہینے تک آئی ایس سے مبینہ وابستگی کے بعد نوجوان نے وطن واپسی کی خواہش ظاہرکی',
 'یوراج سنگھ نے بگ باس کی اس کنٹسٹ پر لگایا دھوکہ دہی کا الزام',
 'پبلک ٹوائلیٹ میں بہن کو کھانا پہنچانے آئی خاتون کا ہوا جنسی استحصال',
 'ایس پی خاتون لیڈر کے ہوٹل میں چل رہا تھا سیکس ریکٹ ، قابل اعتراض حالت میں ملے لڑکے اور لڑکیاں',
 'والد نے اپنی ہی تین بیٹیوں کی عزت تار تار کردی، 9 سالہ بیٹی کو 5 سال سے بنا رہا تھا اپنی ہوس کا شکار',
 'ٹی وی کے ایگزیکٹو پروڈیوسرکو اداکارہ سےعصمت دری کےمعاملہ میں 7 سال کی قید',
 'چاکلیٹ لینے گئی تھی معصوم ، دکاندار نے کی ویڈیو گیم دکھانے کے بہانے نا بالغ بچی سے ریپ کی کوشش ',
 'شرمناک : نابالغ بیٹی کا ریپ کے الزام میں والد گرفتار',
 'سوتیلے باپ نے بیٹی کی آبروریزی کی ، ویڈیو بناکر وائرل کرنے کی دی دھمکی ، کیس درج',
 'حیوانیت کی ساری حدیں پار ، 50 سالہ خاتون کی آبروریزی کی کوشش ، پرائیویٹ پارٹ میں ڈالی لال مرچ',
 'ماں نے اپنی ہی نابالغ بیٹی کو جسم فروشی کے دھندہ میں دھکیلا ، تین کے خلاف معاملہ درج',
 '۔"ممی - پاپا سے دور لے جاکر میرے ساتھ اجتماعی آبروریزی کی ، انہیں پلیز پھانسی دے دو "۔',
 'بھوپال شیلٹر ہوم ریپ معاملہ : آپریٹر کی درندگی سے تین بچیوں کی ہوئی موت',
 "افیم کھلا کر خواتین کی آبروریزی کرتا تھا زانی 'جلیبی' بابا، فحش ویڈیو بناکرکرتا تھا بلیک میل",
 'اسکول کے 4 طلبا نے 10ویں کی طالبہ کا کیا ریپ ، پرنسپل سمیت 9 گرفتار',
 'شام میں النصرہ فرنٹ نے بارہ لوگوں کو سر میں گولی ماری',
 'فرخ آباد : آبروریزی کے بعد نابالغ لڑکی ہوئی حاملہ تو کروا دیا اسقاط حمل ، پھر ہوئی اجتماعی آبروریزی',
 'پارن کی عادت کے سبب نیوی کمانڈر نے پوسٹ کی بیوی کی نیوڈ تصاویر، گرفتار',
 'باپ کے جنسی تشدد سے تنگ آکر بالآخر تین سگی بہنوں نے اسے اتار دیا موت کے گھاٹ',
 'درندگی: دلی میں آٹھ ماہ کی بچی کی عصمت دری، بچی اسپتال میں بھرتی، حالت سنگین',
 'سوتیلے والد نے 14 سال تک کی تھی اس اداکارہ کی عصمت دری، 17 کی عمر میں کروا دیا تھا اسقاط حمل',
 'جموں و کشمیر: سالوں سے بیٹی کی کر رہا تھا عصمت دری، لڑکی نے زہر کھا کر دی جان',
 '! " وہ اجنبی کہہ رہا تھا، " تم ہی تو چاہتی تھیں کہ میں تمہارا ریپ کروں',
 'انتخابی ماحول میں راجستھان میں بی جے پی کارکن کا قتل',
 'مکان میں چل رہا تھا سیکس ریکٹ 3 لڑکیوں سمیت سرسا پولیس نے 6 لوگوں کو کیا گرفتار',
 'پچاس سالہ رکشا چلانے والے نے 9 سالہ لڑکی کی عصمت دری کی، عوام کا شدید احتجاج',
 'آبروریزی کے بعد حاملہ ہوئی 16 سال کی طالبہ ، ملزم مسلسل دے رہا ہے اسقاط حمل کرانے کی دھمکی',
 'گھر میں گھس کر 4 نوجوانوں نے کی خاتون کی اجتماعی عصمت دری، پھر پھندے پر لٹکایا',
 'عورتوں کے خلاف جرائم میں دہلی سرفہرست: وزارت سیاحت',
 '...پیار، سیکس اور دھوکہ: نہیں رکی شوہر کی عیاشی تو ڈپریشن میں گھلتی گئی بیوی اور پھر ہوا یہ',
 'امریکہ کے ایک ہائی اسکول میں فائرنگ میں دو طالبات ہلاک',
 'واشنگٹن کے ایک ہائی اسکول میں فائرنگ، ایک ہلاک، چار زخمی',
 'واٹس ایپ گروپ والے سیکس ریکٹ کا پردہ فاش، کوچنگ سینٹر کے طالب علموں کو بھیجتے تھے فحش تصاویر اور ویڈیو',
 'بیوی کے دفتر میں گھسا شوہر اور چاقو سے گود ڈالا، موقع پر موت',
 'بلند شہر میں 13 سالہ بچی کی اجتماعی عصمت دری ، 7 لوگوں کے خلاف معاملہ درج',
 'ٹرین میں لڑکی سے ریپ : معاملے میں 24 گھنٹے بعد بھی ایف آئی آر درج نہیں ، واقعے کو دبانے کوشش',
 'فحش گانا بجانے سے روکا تو چلتی بس میں خاتون کانسٹیبل کی آبروریزی کی کوشش ، مار پیٹ کی دھمکی',
 'بنگلور: دن دہاڑے بس میں ڈرائیور نے آبروریزی کے بعد لڑکی کو سڑک پر پھینکا',
 'کہانی اس لڑکی کی جو ماڈل تھی، مارڈن کپڑے پہنتی تھی اس لئے اس کا قتل کر ڈالا',
 'نابالغ بچی کی عصمت دری کے بعد قتل، لڑکی کے جسم پر متعدد زخم کے نشانات',
 'حیوانیت کی ہر حد پار، ریپ کے بعد خاتون کے پرائیویٹ پارٹ میں ڈالا اسٹیل کا گلاس ، شوہر گرفتار',
 'چاقو سے حملہ کرکے خاتون ٹیچر کا قتل، پولیس کو شوہر کی تلاش',
 'جب اپنوں نے ہی کیا ریپ ، نابالغ لڑکی نے امتحان کاپی میں لکھی اپنی درد بھری داستاں ، رو پڑیں گے آپ',
 'نشے میں مدہوش شوہر نے رات میں کردی ایسی حرکت کہ نیم عریاں حالت میں ہی تھانہ پہنچ گئی بیوی',
 'امریکہ: اسکول میں طالب علم کی فائرنگ، 2 طلبا ہلاک،17 زخمی',
 'فلائٹ میں 65 سال کے شخص نے خاتون کا کیا جنسی استحصال، مگر ضمانت',
 'خاتون نے خاتون پر لگایا ریپ کا الزام ، لیکن پولیس نہیں کر پا رہی کیس درج',
 'امریکہ: ورجینیا میں 13 سالہ مسلم لڑکی پر حملہ، حجاب پہننے پر کہا دہشت گرد',
 'مظفرپور واقع شیلٹر ہوم میں 21 بچیوں کی عصمت دری کی تصدیق، ایک بچی کو مار کر دفن کرنے کا بھی الزام',
 'انسداد دہشت گردی یونٹ نے گجرات میں 52 لاکھ سے زیادہ نقلی نوٹ برآمد کیا',
 'لڑکی کا 4 سال تک یرغمال بناکر 2 بھائیوں نے کیا ریپ، کئی بار ہوئی حاملہ تو کرادیا اسقاط حمل',
 'بہار: دربھنگہ میں دلت عورت کے ساتھ درندگی، ڈائن بتا کر گاؤں کے لوگوں نے پیٹا',
 'بیس سال چھوٹے لڑکے سے تھے ناجائز تعلقات، شوہر کو مارکر کھیت میں دفنایا',
 'گڑگاؤں میں شادی شدہ عورت سے گینگ ریپ، اغوا کر اراولی پہاڑی لے گئے',
 'الور میں پہلو خان کے بعد ایک اور موب لنچنگ واقعہ، گایوں کی اسمگلنک کے شک میں اکبر کا قتل',
 'مدھیہ پردیش : لڑکی کو اغوا کرکے کی اجتماعی عصمت دری ، متاثرہ کی حالت نازک',
 'آرمی کے 4 جوان معذور خاتون کا چار سالوں تک کرتے رہے ریپ ، ویڈیو کلپ بھی بنائی، مقدمہ درج',
 'دلی: چلتی بس میں طالبہ کے سامنے شخص نے کی گندی حرکت، کسی نے نہیں کی مدد',
 'بنگلورو سے گرفتار ہوئے تین پاکستانی شہری، نیپال کے راستے ہوئی تھی انٹری',
 'گیارہویں جماعت کی طالبہ سے لیا وہاٹس ایپ نمبر اور پھر فحش میسیج بھیجنے لگا پرنسپل ، گرفتار',
 'منگلور جیل میں قیدیوں میں جھگڑا، دو افراد ہلاک، 9 زخمی',
 'مرادآباد میں دلت لڑکی کی عصمت دری ، مندر کا 52 سالہ پجاری گرفتار',
 'انسانیت شرمسار ، نشے میں مدہوش نوجوان نے حاملہ بکری سے کیا ریپ ، ہوئی موت',
 'دلی میں بھیڑ نے چوری کے شک میں نابالغ کا پیٹ پیٹ کر کیا قتل',
 'کار میں شور کرنے پر ماں کے بوائے فرینڈ نے 3 سال کے بچے کے ساتھ کی یہ گندی حرکت، اڑجائیں گے ہوش',
 'امیٹھی کے موہن گنج علاقہ میں حاملہ خاتون کے ساتھ اجتماعی عصمت دری',
 'تین لڑکوں کی ہوس کا شکار بنی 6 سال کی بچی ، ملزموں میں بھائی بھی شامل',
 'ریپ کے بعد حاملہ ہو گئی تھی 5 ویں کی نابالغ طالبہ، ڈاکٹروں نے کرایا اسقاط حمل ',
 'سی سی ٹی وی سے پکڑی گئی ٹی وی اداکارہ کی چوری',
 'تقریبا 12 سال سے کوما میں رہ رہی خاتون نے بچےکو دیا جنم، معاملہ کی تحقیقات جاری',
 'وہاٹس ایپ پر چل رہے سیکس ریکٹ پر چھاپہ، پکڑی گئیں شادی شدہ خواتین',
 'مہندی سے ہاتھ میں سوسائڈ نوٹ لکھ کر دوسری منزل سے کود کرلڑکی نے کی خود کشی',
 'شوہر نے بیوی پر بنایا اسقاط حمل کا دباؤ ، انکار کرنے پر باپ نے کردیا 3 ماہ کی بیٹی کا قتل',
 'پارلر سے گھر جا رہی لڑکی کو راستے میں چار نوجوانوں نے اغوا کرکے کی اجتماعی عصمت دری',
 'آنر کلنگ: بھائی نے بہن اور اس کے عاشق کو پھاوڑے سے کاٹا',
 'بیٹے کو بچانے کے لئے ماں کو کرنی پڑی جسم فروشی، رشتہ داروں نے کسٹمرس بن کر کیا یہ کام',
 'بہار: انسانیت پھر شرمسار، 7 سالہ بچی کی عصمت دری کی، سڑک پر تڑپتا چھوڑ ملزم فرار',
 'بلند شہر : برگر کے بہانے دو معصوم بہنوں کی آبروریزی کرنے والا ملزم گرفتار',
 'حیدرآباد: نشیلی اشیا دے کر لڑکی کی اجتماعی عصمت دری، متاثرہ کی تصویر انٹرنیٹ پر آنے کے بعد معاملہ درج',
 'موبائل پر رات بھر ویڈیوز دیکھ رہی تھی بیوی ، شوہر نے اٹھایا یہ خوفناک قدم ، جان کر اڑجائیں گے ہوش',
 'نابالغ لڑکی کا دو لوگوں نے ریپ کرکے بنایا ویڈیو، حاملہ ہونے پر ہوا انکشاف',
 'امریکہ میں شدت پسند عناصرنے مسجد کو آگ لگائی، اسلامک سینٹر آف ایسٹ کا نصف سے زیادہ حصہ تباہ',
 'بارہویں کی طالبہ سے اجتماعی عصمت دری ، ہاتھ منھ باندھ کر تالاب میں پھینکا ، 6 گرفتار',
 'نشیلی دوا کھلا کرنابالغ لڑکی کی اسپتال میں کی اجتماعی عصمت دری ، ملزم گرفتار',
 'ہریانہ میں عورت کے ساتھ گینگ ریپ، شناخت مٹانے کے لیے چہرے کو گاڑی سے کچلا',
 'غازی آباد میں انجینئرنگ کالج کے ڈائریکٹر پر دست درازی کا معاملہ درج',
 'سیکس سے انکار کیا تو ماڈل کا کر دیا قتل، ماڈل کے پرائویٹ پارٹ پر ملے چوٹ کے نشان',
 'بنگلور میں چلتی گاڑی میں گارمنٹ فیکٹری کی ملازمہ کی عصمت دری',
 'کرایہ کا مکان تلاش کر رہی داروغہ کی اہلیہ کی طمنچہ کے نوک پر آبروریزی ، ملزم گرفتار',
 'اترپردیش میں پھر سرعام ایک صحافی کو گولی مار کر قتل',
 'شمالی کشمیر: جوان پر میجر کے قتل کا الزام، موبائل کو لے کر ہوا تھا تنازعہ',
 'بلوچستان میں بندوق برداروں کی فائرنگ میں ہزارہ برادری کے چار افراد جاں بحق',
 'اجتماعی عصمت دری: خاتون کو یرغمال بناکر 40 لوگوں نے 4 دنوں تک کیا ریپ ، نشے کے انجیکشن بھی دئے',
 'بی جے پی رکن اسمبلی پریم لتا کا متنازعہ بیان: ریپ کے لئے بے روزگار بچے ذمہ دار',
 'بہار میں ایک اور صحافی کا گولی مار کر قتل، موٹر سائیکل سواروں نے گھیر کر ماریں سات گولیاں',
 'موبائل آرڈر کرنے پر ملا صابن ، ایمیزون کے کنٹری ہیڈ پر کیس درج',
 'دوا لینے آئی 26 سالہ بیوہ کا اغوا کرکے کیا ریپ، دو نوجوانوں پر معاملہ درج',
 'ممبئی : سوشل میڈیا کی دوستی کا دردناک انجام ، جنسی تعلقات سے انکارپر 20 سالہ لڑکی کا بے رحمانہ قتل',
 "ہیرا تاجرکے اغوا معاملے میں پھنس گئی ادا کارہ 'گوپی بہو'، پولیس نے حراست میں لیا",
 'خوشحالی کیلئے گھر پر بلایا تانترک ، خاتون کی آبروریزی کرکے ہوا فرار',
 'لکژری ٹرینوں کے لائق نہیں ہم؟ مسافرنل اور پائیدان تک چرا لے گئے',
 'بیوی اور دو بچوں کو موت کے گھاٹ اتار کر تھانے پہنچا سپاہی',
 'باپ کررہا تھا نابالغ بیٹی کا کئی سالوں سےجنسی استحصال ، حاملہ ہو گئی بچی',
 'دن دہاڑے ایڈیشنل سیشن جج کی بیوی ۔ بیٹے کو ان کی سکیورٹی میں تعینات سرکاری محافظ نےماری گولی',
 'حیدر آباد : برہم عاشق نے سر راہ مٹی کا تیل ڈال کر لگادی آگ ، اسپتال میں دوران علاج 25 سالہ لڑکی کی موت',
 'چھیڑ چھاڑ کی مخالفت کی تو گاوں والوں نے اسکول میں گھس کر کی پٹائی ، 34 طالبات زخمی',
 'فری لانس کال گرلس کا ریکٹ بنا چکی تھی جنسی تجارت کی لیڈی ڈان',
 'دہلی کے مانسروور پارک میں ایک ہی کنبہ کی چار خواتین سمیت پانچ افراد کا قتل',
 'نیویارک میں کورٹ میرج کے بعد معشوقہ کا بھوپال میں قتل، لاش کو گھر میں دفنایا',
 '! خاتون کے ساتھ جنسی تعلقات بنا رہا تھا بزرگ، اچانک نکلا کیمرا اور گنوانی پڑی جان',
 'گنے کے کھیت میں 14 سالہ لڑکی کی اجتماعی آبروریزی ، پکڑے جانے کے ڈر سے بعد میں کردیا قتل',
 'باپ نے اپنی ہی نابالغ بیٹی کو کیا چارمرتبہ حاملہ ، نشیلی اشیا کھلا کر دوسال سے کررہا تھا ریپ',
 'دیوریا میں دوشیزہ کی اجتماعی آبروریزی، پانچ افراد کے خلاف مقدمہ درج',
 '30 سال پہلے 8 سالہ بچی کی عصمت دری کے بعد کیا تھا قتل',
 'دہلی کے کناٹ پلیس پر پڑی چوروں کی نظر، اڑائیں 1.30 کروڑ روپے کی گھڑیاں',
 'دوا دلانے کے بہانے چار دن تک بچی کی عصمت دری کرتا رہا خالو ',
 'دوستوں کے ساتھ سوئمنگ پول میں نہانے گئی تھی لڑکی، 8 لڑکوں نے 10 مرتبہ مسلسل کیا ریپ اور ویڈیو بھی بنایا',
 'بوڑھے تانترک نے لڑکی کی عصمت دری کی',
 'عصمت دری کی شکار لڑکی کی ماں نے پولیس کے سامنے ملزم کی کر دی پٹائی',
 'ہماچل پردیش : بہو کا الزام ، جب بھی گھر آتا تھا سسر ، نشیلی اشیا کھلا کر کرتا تھا آبروریزی ، کیس درج',
 'عصمت دری کے بعد لڑکی کا گلا کاٹ کر کیا قتل',
 'سواتی قتل کیس:کلیدی ملزم گرفتار، گرفتاری کے وقت خود کا گلا کانٹنے کی کوشش کی تھی',
 'بائک خراب ہونے پر لڑکیوں نے فون کرکے دوست سے مانگی مدد ، وہاں پہنچے 11 لڑکوں نے کیا ریپ',
 'بستر میں آدیواسی کے قتل کے معاملہ میں ڈی یو اور جے این یو کی پروفیسرز پر مقدمہ درج',
 'بنگلورو : اولا کیب ڈرائیور نے خاتون مسافر کے ساتھ چھیڑ چھاڑ کی ، کپڑے اتروا کر کھینچی تصویر',
 'کینیا میں اسکول میں فائرنگ، چھ بچوں سمیت سات کی موت',
 'بی جے پی ممبر پارلیمنٹ کنور سنگھ تنور کے بیٹے کو کال گرل نے لوٹ لیا',
 'مدھیہ پردیش : آٹھویں کلاس کی 15 سالہ نابالغ لڑکی کو اجتماعی آبروریزی کے بعد زندہ جلا دیا گیا',
 'شوہرکاعضو خاص کاٹ کرعاشق کے ساتھ فرار ہوئی بیوی',
 'نابالغ بچی کا بیٹے نے کیا ریپ و قتل، باپ نے بیٹے کے ساتھ مل کر جو کیا جان کر اڑ جائیں گے ہوش',
 'باہر سے لڑکیاں بلا کر اپنے ہی گھر میں دھندا کرواتی تھیں ساس۔ بہو، پولیس نے ایسے کیا گرفتار',
 'فون کرکے 15 سالہ نابالغ طالبہ کو گھر سے باہر بلایا ، سنسان جگہ لے جا کر کی آبروریزی ، گرفتار',
 'نئے لڑکے سے رشتہ رکھنا چاہتی تھی معشوقہ ، پرانے عاشق کے ساتھ رچی ایسی خوفناک سازش کہ جان کر اڑجائیں گے ہوش',
 'سوتیلی ماں نے 4 سال کوٹھری میں رکھا بند اور کھانے کو دی سوکھی بریڈ، مرنے کی حالت میں پہنچا بچہ',
 'نشہ کے عادی بدمعاشوں نے پوچھا : گانجا کہاں ملتا ہے ؟ دوکاندار نے نہیں بتایا تو ماردی گولی',
 'خاتون کو اغوا کرکے 4 لوگوں نے کی اجتماعی عصمت دری ، ہائی وے پر پھینک کر فرار',
 'مدھیہ پردیش میں بچیوں کے جسمانی استحصال کے الزام میں آشرم منتظمہ گرفتار',
 'مظفر نگر میں بھارتیہ جنتا پارٹی کے لیڈر راجا بالمیكی کا گولی مار کر قتل',
 'کانپور میں سب انسپکٹر کی بیٹی کے ساتھ اجتماعی آبروریزی، ملزمان گرفتار',
 'جسمانی تعلقات بنانا چاہتا تھا باپ ، بیٹی نے کی مخالفت تو کردی یہ وحشیانہ حرکت',
 'شرمناک! لدھیانہ میں لڑکی کو کار سے کھینچ کر 10 لوگوں نے کیا گینگ ریپ',
 'ماں کو غیرمرد کے ساتھ قابل اعتراض حالت میں دیکھا تو آپا کھو بیٹھا نابالغ بیٹا، کر دیا یہ کام',
 'ایک 47 سال کا شخص 13 سال کی بچی کا ایک مہینے تک کرتا رہا ریپ، ایسے ہوا انکشاف',
 'شوہر عہدے کی خاطر غیر مردوں کے ساتھ کرواتا تھا بیوی سے ہمبستری ، کانگریس لیڈر پر بیوی کا الزام',
 'دونوں ایک دوسرے سے کرتی تھیں بے حد محبت، پھر کچھ ایسا ہوا کہ ایک نے کر لی خود کشی اور دوسری نے پی لیا زہر',
 'بہار میں طالبہ سے 18 نے کیا ریپ، پرنسپل اور 2 اساتذہ سمیت 15 طلباء پر کیس',
 'جلیبی- سموسا بیچنے والا ایسے بنا بلّو بابا، 120 خواتین کی کرچکا ہے آبروریزی',
 'آن لائن سیکس ریکٹ کا پردہ فاش ، کنڈوم اور قوت بخش ادویات برآمد',
 'حیوانیت کی ساری حدیں پار ، پٹنہ میں 7 ماہ کی بچی کی آبروریزی ، حالت سنگین ، ملزم گرفتار',
 'ویڈیو میں بندر کے پرائیویٹ پارٹ سے کھیلتی دیکھی خاتون، تین سال کی سزا',
 'نوکری نہ ہونے پر لڑکی نے کیا بریک اپ تو غصہ میں بوائے فرینڈ نے کردی یہ وحشیانہ حرکت',
 'نابالغ لڑکی کا 55 سالہ شخص نے کر دیا ریپ ، 5 ماہ سے حاملہ ہے بچی',
 'دہلی : ڈیفنس کالونی میں نشہ میں دھت خاتون نے اپنی ماں اوربھائی کو ماری گولی ، ملزمہ گرفتار، تفتیش شروع',
 'معصوم بچی سے عصمت دری کرکے بلیڈ سے بازووں پرکیا 50 بارحملہ، 4 ملزمین کو گرفتار کرنے میں پولیس کامیاب',
 'آبروریزی کرنے میں ناکام ہوا تو بھابھی کے ساتھ کردی یہ وحشیانہ حرکت ، پھر اٹھایا یہ انتہائی قدم',
 'بہار: آر جے ڈی لیڈر محمد شہاب الدین کے بھتیجے کا گولی مار کر قتل',
 'نا بالغ لڑکی کی اجتماعی عصمت دری : پہلے اغوا کیا پھربعد میں یرغمال بناکر 8 دنوں تک کیا ریپ',
 'لڑکی کا الزام: داتی مہاراج اور اس کے شاگردوں نے کی بار بار عصمت دری',
 'بیوی نے کس کرتے وقت کاٹ دی شوہر کی زبان، کہا- بلکل بھی اچھا نہیں تھا وہ',
 'غیر مرد کے ساتھ بیوی کا پورن ویڈیو و تصویریں کار میں کرتے تھے دونوں دپورن فلم جیسی حرکتیں',
 'جھارکھنڈ : پنچایت کے فیصلہ سے ناراض عاشق نے اجتماعی آبروریزی کی متاثرہ معشوقہ کو زندہ جلایا',
 'پڑوسی نے 8 سالہ معصوم بچی کی بنایا اپنی حیوانیت کا شکار، کانوڑ یاتر پر گئے تھے اہل خانہ',
 'دہلی میں چلتی ٹرین میں عورت سے کی عصمت دری، ملزم گرفتار، مقدمہ درج',
 'گھر میں گھس کر ماں کے پاس سوئی بیٹی کو اٹھا کر لے گئے ، 8 لوگوں نے کیا ریپ',
 'ممبئی کے ڈانس بارمیں چھاپہ، 11 گاہک پولیس حراست میں، 7 لڑکیاں بھی برآمد',
 'جنسی تعلقات قائم کرنے کے لئے مسلسل دباو ڈالتی تھی خاتون، پریشان ہو کر نوجوان نے لگائی پھانسی',
 'شرمناک: تین سال کی معصوم بچی کا آبروریزی کے بعد قتل، ملزم فرار',
 'امیٹھی میں ایک ہی خاندان کے 11 افراد کا قتل، پولیس تفتیش میں مصروف',
 'سیکس کے بعد طوائفوں کو اتار دیتا تھا موت کے گھاٹ',
 'ممبئی : خاتون ماڈل سے بد تمیزی کرنے اور دھمکی دینے کے الزام میں دو افراد گرفتار ، پوچھ گچھ جاری',
 'بدمعاشوں نے کی حیوانیت کی حدیں پار، ریپ کے بعد نکالی لڑکی کی آنکھیں',
 'نوئیڈا: چلتی گاڑی میں خاتون کی اجتماعی آبروریزی ، بدمعاشوںنے ایم ایم ایس بھی بنایا',
 'تین سال کی بچی سے ریپ اور دو بہنوں کے قتل میں گرودوارے کے 55 سال کے سیوا دار کو سزائے موت',
 'فیس بک پر اپنی ہی بھانجی کی بنائی فرضی آئی ڈی اور وائرل کردیں فرضی فحش تصویریں ، گرفتار',
 'بی جے پی کے سابق ایم ایل اے نے لڑکی کو بیہوش کرکے کیا ریپ ، معاملہ درج',
 'دنیا کا پہلا کرکٹر، جسے دی گئی تھی پھانسی کی سزا، کچھ اس طرح کیا تھا بیوی کا قتل',
 'تین طلاق کے بعد سسر کے ساتھ حلالہ، عدت میں شوہر نے کیا ریپ',
 'جموں۔ کشمیر: 9 سالہ بچی کی اجتماعی عصمت دری کے بعد آنکھیں نکال کر جسم پر چھڑکا تیزاب',
 'بھائی اور دوستوں نے مل کر کی نابالغ بہن کی اجتماعی عصمت دری، تمام ملزمین فرار',
 'اترپردیش : دوسری شادی کرنے کیلئے شوہر نے پانچ ماہ کی حاملہ بیوی کو فون پر دیا تین طلاق',
 'کشمیر میں گرام پردھان کا گولی مار کر قتل',
 'گیسٹ ہاوس میں چل رہا تھا بڑا سیکس ریکیٹ ، پولیس نے کیا اس طرح پردہ فاش ، کئی گرفتار',
 'بارہ دن تک اغواکاروں کے قبضہ میں رہا معصوم دلشاد گارڈن سے بحفاظت رہا',
 'چھتیس گڑھ میں سینئر کانگریس لیڈر کی ماں کا قتل، باپ بھی شدید زخمی',
 'ملازمت سے محروم شخص نے خودکشی کی',
 'ہریانہ: گروکل کے 6 بچوں کا الزام – سال بھر سے بدفعلی کر رہے ہیں سینئر طلباء',
 'غازی آباد : مدرسہ میں لے جاکر ایک نابالغ نے 10 سالہ لڑکی کی آبروریزی کی ، ملزم گرفتار، لوگوں کا احتجاج',
 'حیدرآباد : فیس بک پر کی دوستی ، پھر سینما ہال لے جا کر لڑکی کی کردی آبروریزی ، گرفتار',
 'رشتہ ہوا شرمسار: بھائی نے ہی لوٹ لی 6 سالہ بہن کی عزت',
 'پاکستان میں 15 سال کی لڑکی سے ایمبولینس میں ریپ، سڑک پر پھینک کر ملزم فرار',
 'گریٹر نوئیڈا اجتماعی عصمت دری معاملے میں تین افراد گرفتار',
 'گنگا میں نہا رہی خاتون سے اجتماعی عصمت دری کے ملزم پر ہو سکتا ہے بڑا حملہ',
 'نہیں رک رہے ریپ کے معاملے بہار میں 5 سالہ بچی کی اجتماعی آبروریزی',
 'پٹنہ : بدمعاشوں نے کامرس کالج میں مچایا ہنگامہ ، پروفیسروں کی گاڑیوں سمیت پرنسپل آفس کو بنایا نشانہ',
 'شرمناک: قبر سے نكال کر لڑکی کی لاش کے ساتھ جنسی تعلقات قائم کیا',
 'کنہیا کمارکے لئے دھماکہ خیز مادہ سے بھرا پارسل اور دھمکی آمیز خط برآمد',
 'اپنی ہی اہلیہ کے ساتھ سیکس کے دوران پارن چلاکر یہ کام کرتا تھا شوہر،جان کر اڑ جائیں گے ہوش',
 'دہلی: گھر سے باہر گئے تھے والدین، بڑے بھائی نے کی 8 سالہ بہن کی عصمت دری، بچی کی حالت نازک',
 'شرمناک : آئی سی یو میں آکسیجن لگی نابالغ لڑکی کا اسپتال کے اسٹاف نے نشے کے انجکشن دیکرکیا ریپ',
 'بین ریاستی سیکس ریکیٹ کا پردہ فاش ، نوکری کا جھانسہ دے کر لڑکیوں کو جسم فروشی پر کیا جاتا تھا مجبور',
 'مکھیا نے دوستوں کے ساتھ مل کر نابالغ لڑکی کو اغوا کیا ، پھرجبرا شراب پلاکر کی اجتماعی آبروریزی ، گرفتار',
 'پاکستان: 18 سالہ بس ہوسٹس مہویش کا قتل، جانیں کیا تھا اصل معاملہ',
 'لندن میں چاقو سے حملہ کرنے والا ذہنی مریض، حملہ میں ایک خاتون ہلاک ، 5 افراد زخمی',
 'نئے سال کی پارٹی میں 11 افراد کے قتل کے بعد حملہ آور نے خود کو ماری گولی',
 'چلتی ٹرین میں نابالغ لڑکی کی اجتماعی عصمت دری، فوج کے دو جوانوں پر الزام',
 'دسویں کی طالبہ کی اجتماعی آبروریزی کرکے بنایا فحش ویڈیو ، پھر سوشل میڈیا پر کردیا وائرل ، گرفتار',
 'منالی میں 33 سالہ غیر ملکی خاتون کی اجتماعی عصمت دری ، ایف آئی آر درج',
 'خواتین کا جنسی استحصال کر کے ویڈیو بناتا تھا یہ ڈھونگی بابا ، کئی کلپ بر آمد',
 'بہار کے دربھنگہ میں دو انجینئرز کے قتل سے سنسنی، اپوزیشن نے کہا جنگل راج کی واپسی',
 'اس 63 سال کے بزرگ کی 5 معشوقائیں ، پیسے کی وجہ سے پہلی نے چھوڑا تو کرنے کگا یہ کام۔۔',
 'شرمناک : سگے بھائی پر 12 سال کی معصوم بہن کی آبروریزی کا لگا الزام ، ملزم فرار',
 'لکھنئو میں طالبات کے استحصال کے الزام میں مدرسے کا منتظم گرفتار',
 'مغربی بنگال میں 100 سال کی سن رسیدہ خاتون کی آبروریزی ، ملزم گرفتار',
 'گورکھپور: ڈاکٹر کفیل خان کے بھائی کو بائیک سوار بدمعاشوں نے ماری گولی',
 'سوتن کے ساتھ رہنے پر خاتون نے شوہر کا پرائیویٹ پارٹ کاٹ ڈالا',
 'بندوق کی نوک پر کینسر متاثرہ کے ساتھ 3 سال تک جنسی استحصال کرتا رہا پولیس اہلکار',
 'ہے پربھو! روپے پیسے اور موبائل چھین کر لڑکی کو چلتی ٹرین سے پھینک دیا',
 'فرانس کے شہر ایوگنن میں مسجد کے قریب فائرنگ میں آٹھ افراد زخمی',
 'امریکی کلینک میں فائرنگ ، تین ہلاک نو زخمی',
 'شوہر غیر مردوں سے ہمبستر ہونے کا بناتا تھا دباؤ ، بیوی کے انکار کرنے پر دیا طلاق',
 'گیارہ سالہ بچی کی 7 مہینے تک عصمت دری: پولیس نے 18 لوگوں کو کیا گرفتار',
 'بڑے سیکس ریکٹ کا پردہ فاش ، قابل اعتراض اشیا برآمد، پانچ لڑکیوں سمیت چھ گرفتار',
 'چنئی : برقع میں آئی ایک خاتون نے مسجد کے پاس امام کو زندہ جلایا ، کیس درج ، جانچ شروع',
 'آٹھویں جماعت کی طالبہ نے اسکول میں بیٹی کو دیا جنم ، پھراسکول نے کیا کچھ ایسا کہ حیران ہوجائیں گے آپ',
 'مدھیہ پردیش : نان ویج نہیں بنانے پر بیوی کو زندہ جلایا',
 'علی گڑھ مسلم یونیورسٹی کے طالب علم کے قتل کےبعد حالات کشیدہ',
 'گوپال گنج میں شو سے لوٹ رہی نابالغ آرکیسٹرا ڈانسر کی آبروریزی ، ملزم گرفتار',
 'سوئٹزرلینڈ میں مسجد میں فائرنگ ، تین افراد زخمی، دو کی حالت نازک، حملہ آور فرار',
 'دہلی ۔علی گڑھ شاہراہ پر بدمعاشوں کی درندگی، گھر والوں کے سامنے ماں بیٹی سے گینگ ریپ',
 'اسپورٹس اکیڈمی کی غیر شادی شدہ 19 سالہ کھلاڑی نے بچی کو دیا جنم ، وزارت میں مچی افراتفری',
 'بندوق کی نوکر پر آبروریزی کرکے بنائی فحش ویڈیو ، پھر کرتا رہا بلیک میل ، متاثرہ طالبہ نے اٹھایا یہ خوفناک قدم',
 'اب یوپی کے دیوریا میں مظفرپور جیسے سیکس ریکٹ کا پردہ فاش، 24 خواتین آزاد کرائی گئیں',
 'محض 16 سال کی عمر تک کتنی مرتبہ ہو چکا تھا اس کے جسم کا سودہ ، پڑھیں رونگٹے کھڑے کر دینے والی یہ کہانی',
 'حیوانیت کی ساری حدیں پار ، راجستھان میں دلت نواجوان کا پیٹ پیٹ کر بے رحمی سے قتل ، تین حراست میں ',
 'بلیو فلموں کے دم پر بیوی کو اس طرح بلیک میل کر رہا تھا شوہر',
 'پورن دکھا کر سننے اور بولنے سے معذورلڑکیوں کی کرتا تھا عصمت دری',
 'راجستھان میں بھیڑ کی وحشیانہ انداز میں پٹائی سے ایک اور مسلم مزدور کی موت',
 'سنسنی خیز ! 9 سال کے بچے سے مہینوں تک ریپ کرتی رہی 36 سالہ خاتون ، ایف آئی آر درج',
 'بہار : ٹیوشن کیلئے گئی 8 سالہ بچی کو ٹیچر نے بنایا اپنی ہوس کا شکار ، ثبوت مٹانے کیلئے قتل کی بھی کوشش',
 'معصوم بچیوں کو بناتا تھا اپنی حیوانیت کا شکار ، ملی 16 مرتبہ پھانسی کی سزا',
 'دھرمشالہ میں اس بولڈ اداکارہ کے ساتھ کی گئی چھیڑ چھاڑ ، کچھ اس طرح بیان کیا اپنا درد',
 'زبردستی شراب پلا کر دلت نابالغ لڑکی کی اجتماعی آبروریزی ، کیس درج',
 'فلیٹ میں 8 بچیوں کو یرغمال بنا کر گرم چاقو سے داغتی تھی خاتون:گوا پولیس',
 'گیارہ سال کی عمر میں طلاق مانگ رہی ہے کیوں کہ وہ پھر جانا چاہتی ہے اسکول',
 'باغپت میں کہا سنی کے بعد پولیس اہلکار نے نوجوان کو ماری گولی',
 'دہلی : خود کو ناگا بابا بتانے والے روی گری کو پولیس نے چھیڑ چھاڑ کے الزام میں کیا گرفتار',
 'الیکشن میں ہار کا بدلہ لینے کے لئے خاتون بی ڈی سی کی بیٹی کی آبروریزی ، متاثرہ نے لگائی پھانسی',
 'نو سال کی نابالغ بچی کی اجتماعی آبروریزی ، ملزمین میں اسکول وین کا ڈرائیور بھی شامل',
 'نشہ کی گولیاں سونگھا کر بی اے تھرڈ ائیر کی طالبہ کی کرتے تھے آبروریزی ، سہیلی بھی دیتی تھی ملزمین کا ساتھ',
 'شادی کا جھانسہ دے کر لڑکی کی کرتا رہا عصمت دری، پھر چھوڑ دیا ساتھ',
 'یوکرین میں تین ہندستانی طلبہ پر چاقوؤں سے حملہ، دو ہلاک، تیسرا شدید طور پر زخمی',
 'مغربی بنگال کی لڑکی کی جمشید پور میں اجتماعی آبروریزی ، عاشق نے مار پیٹ کرکے ندی کے کنارے پھینکا',
 'مساج پارلر کی آڑ میں چل رہا تھا جسم فروشی کا کاروبار،11 لڑکیوں سمیت 15 گرفتار']

print(urdunews)

['نہاتی ہوئی خاتون کا ویڈیو بنا رہا تھا آئی آئی ٹی طالب علم ، موبائل میں ملے کئی فحش کلپ', 'کنساس شوٹنگ کے بعد اب جنوبی کیرولینا میں ایک ہند نژاد تاجر کا قتل', 'ایل جے پی لیڈر کے قتل کیس میں 4 لوگوں کے خلاف نامزد ایف آئی آر درج', 'اداکارہ کنگنا رناوت کے ہیئر اسٹائلسٹ نے کیا انتہائی گھنونا کام ، ممبئی پولیس نے سیٹ سے ہی کیا گرفتار', 'محبت سے شروع ہوا رشتہ شادی تک پہنچا ، مگر پھر کچھ ایسا ہوا کہ یکایک سب کچھ بدل گیا', 'راکھی باندھنے کیلئے آئی نابالغ لڑکی کو دو دنوں تک یرغمال بناکر لڑکے نے کی آبروریزی ، کیس درج', 'کٹھوعہ کیس: ملزمین کے وکیل کا متنازعہ بیان "خاتون ہے کتنا دماغ ہو گا اس میں"۔', 'دہلی : آبروریزی کے مجرم اوبیر کیب کے ڈرائیور شیو کمار کو عمر قید کی سزا', 'پونے : یتیم خانہ میں جنسی استحصال کے الزام میں مولوی گرفتار، آزاد کرائے گئے 36 بچے', 'ٹیچر نے بھیجیں 13 سال کےاسٹوڈینٹ کو پورن ویڈیوز، ماں نے اس ایپ کے ذریعے کیا سنسنی خیز انکشاف', 'گروگرام: دیوالی پر پہلے باپ کے ساتھ شراب پی، پھر اپنی ہی بہن کا کیا ریپ', 'عاشق جوڑے کے ساتھ شرپسند عناصر نے کی چھیڑ چھاڑ ، لڑکی کا اتروا

In [5]:
urdu_stop_words = ["آئی","آئے","آج","آخر","آخرکبر","آدهی","آًب","آٹھ","آیب","اة","اخبزت","اختتبم","ادھر","ارد","اردگرد","ارکبى","اش","اضتعوبل","اضتعوبلات","اضطرذ","اضکب","اضکی","اضکے","اطراف","اغیب","افراد","الگ","اور","اوًچب","اوًچبئی","اوًچی","اوًچے","اى","اً","اًذر","اًہیں","اٹھبًب","اپٌب","اپٌے","اچھب","اچھی","اچھے","اکثر","اکٹھب","اکٹھی","اکٹھے","اکیلا","اکیلی","اکیلے","اگرچہ","اہن","ایطے","ایک","ب","ت","تبزٍ","تت","تر","ترتیت","تریي","تعذاد","تن","تو","توبم","توہی","توہیں","تٌہب","تک","تھب","تھوڑا","تھوڑی","تھوڑے","تھی","تھے","تیي","ثب","ثبئیں","ثبترتیت","ثبری","ثبرے","ثبعث","ثبلا","ثبلترتیت","ثبہر","ثدبئے","ثرآں","ثراں","ثرش","ثعذ","ثغیر","ثلٌذ","ثلٌذوثبلا","ثلکہ","ثي","ثٌب","ثٌبرہب","ثٌبرہی","ثٌبرہے","ثٌبًب","ثٌذ","ثٌذکرو","ثٌذکرًب","ثٌذی","ثڑا","ثڑوں","ثڑی","ثڑے","ثھر","ثھرا","ثھراہوا","ثھرپور","ثھی","ثہت","ثہتر","ثہتری","ثہتریي","ثیچ","ج","خب","خبرہب","خبرہی","خبرہے","خبهوظ","خبًب","خبًتب","خبًتی","خبًتے","خبًٌب","خت","ختن","خجکہ","خص","خططرذ","خلذی","خو","خواى","خوًہی","خوکہ","خٌبة","خگہ","خگہوں","خگہیں","خیطب","خیطبکہ","در","درخبت","درخہ","درخے","درزقیقت","درضت","دش","دفعہ","دلچطپ","دلچطپی","دلچطپیبں","دو","دور","دوراى","دوضرا","دوضروں","دوضری","دوضرے","دوًوں","دکھبئیں","دکھبتب","دکھبتی","دکھبتے","دکھبو","دکھبًب","دکھبیب","دی","دیب","دیتب","دیتی","دیتے","دیر","دیٌب","دیکھو","دیکھٌب","دیکھی","دیکھیں","دے","ر","راضتوں","راضتہ","راضتے","رریعہ","رریعے","رکي","رکھ","رکھب","رکھتب","رکھتبہوں","رکھتی","رکھتے","رکھی","رکھے","رہب","رہی","رہے","ز","زبصل","زبضر","زبل","زبلات","زبلیہ","زصوں","زصہ","زصے","زقبئق","زقیتیں","زقیقت","زکن","زکویہ","زیبدٍ","صبف","صسیر","صفر","صورت","صورتسبل","صورتوں","صورتیں","ض","ضبت","ضبتھ","ضبدٍ","ضبرا","ضبرے","ضبل","ضبلوں","ضت","ضرور","ضرورت","ضروری","ضلطلہ","ضوچ","ضوچب","ضوچتب","ضوچتی","ضوچتے","ضوچو","ضوچٌب","ضوچی","ضوچیں","ضکب","ضکتب","ضکتی","ضکتے","ضکٌب","ضکی","ضکے","ضیذھب","ضیذھی","ضیذھے","ضیکٌڈ","ضے","طرف","طریق","طریقوں","طریقہ","طریقے","طور","طورپر","ظبہر","ع","عذد","عظین","علاقوں","علاقہ","علاقے","علاوٍ","عووهی","غبیذ","غخص","غذ","غروع","غروعبت","غے","فرد","فی","ق","قجل","قجیلہ","قطن","لئے","لا","لازهی","لو","لوجب","لوجی","لوجے","لوسبت","لوسہ","لوگ","لوگوں","لڑکپي","لگتب","لگتی","لگتے","لگٌب","لگی","لگیں","لگے","لی","لیب","لیٌب","لیں","لے","ه","هتعلق","هختلف","هسترم","هسترهہ","هسطوش","هسیذ","هطئلہ","هطئلے","هطبئل","هطتعول","هطلق","هعلوم","هػتول","هلا","هوکي","هوکٌبت","هوکٌہ","هٌبضت","هڑا","هڑًب","هڑے","هکول","هگر","هہرثبى","هیرا","هیری","هیرے","هیں","و","وار","والے","وٍ","ًئی","ًئے","ًب","ًبپطٌذ","ًبگسیر","ًطجت","ًقطہ","ًو","ًوخواى","ًکبلٌب","ًکتہ","ًہ","ًہیں","ًیب","ًے","ٓ آش","ٹھیک","پبئے","پبش","پبًب","پبًچ","پر","پراًب","پطٌذ","پل","پورا","پوچھب","پوچھتب","پوچھتی","پوچھتے","پوچھو","پوچھوں","پوچھٌب","پوچھیں","پچھلا","پھر","پہلا","پہلی","پہلےضی","پہلےضے","پہلےضےہی","پیع","چبر","چبہب","چبہٌب","چبہے","چلا","چلو","چلیں","چلے","چکب","چکی","چکیں","چکے","چھوٹب","چھوٹوں","چھوٹی","چھوٹے","چھہ","چیسیں","ڈھوًڈا","ڈھوًڈلیب","ڈھوًڈو","ڈھوًڈًب","ڈھوًڈی","ڈھوًڈیں","ک","کئی","کئے","کب","کبفی","کبم","کت","کجھی","کرا","کرتب","کرتبہوں","کرتی","کرتے","کرتےہو","کررہب","کررہی","کررہے","کرو","کرًب","کریں","کرے","کطی","کل","کن","کوئی","کوتر","کورا","کوروں","کورٍ","کورے","کوطي","کوى","کوًطب","کوًطی","کوًطے","کھولا","کھولو","کھولٌب","کھولی","کھولیں","کھولے","کہ","کہب","کہتب","کہتی","کہتے","کہو","کہوں","کہٌب","کہی","کہیں","کہے","کی","کیب","کیطب","کیطرف","کیطے","کیلئے","کیوًکہ","کیوں","کیے","کے","کےثعذ","کےرریعے","گئی","گئے","گب","گرد","گروٍ","گروپ","گروہوں","گٌتی","گی","گیب","گے","ہر","ہن","ہو","ہوئی","ہوئے","ہوا","ہوبرا","ہوبری","ہوبرے","ہوتب","ہوتی","ہوتے","ہورہب","ہورہی","ہورہے","ہوضکتب","ہوضکتی","ہوضکتے","ہوًب","ہوًی","ہوًے","ہوچکب","ہوچکی","ہوچکے","ہوگئی","ہوگئے","ہوگیب","ہوں","ہی","ہیں","ہے","ی","یقیٌی","یہ","یہبں"]
filtered_sentence = []
nlp = spacy.blank('ur')

def filterSentence(headline):
    word_tokens = nlp(headline)
    return [w.text for w in word_tokens if not w.text in urdu_stop_words]
    

#     for w in word_tokens:
#         if w.text not in urdu_stop_words:
#             filtered_sentence.append(w.text)
            
for i, headline in enumerate(mydata['Headline']):
    filtered_sentence.append(' '.join([str(elem) for elem in filterSentence(headline)]))
# print(data['Headline'])
print(filtered_sentence)

['ننکانہ صاحب میں بھائی نے 2 بہنوں کو قتل کردیا', 'سوتیلے بیٹے ہاتھوں ماں قتل میں سنسنی خیز انکشاف', 'لاہور میں پولیس اہلکار نے لین دین تنازع کراچی نوجوان کو قتل کردیا', 'لاہور میں پولیس کارروائی ، ڈکیت میاں بیوی گرفتار', 'پتوکی میں ڈاکوؤں 15 سالہ لڑکی سے زیادتی ، وزیراعلی لڑکی گھر پہنچ', 'راولپنڈی میں فائرنگ سے بچے قتل', 'کراچی میں زنجیروں سے بندھی بچی بازیاب ، ماں سوتیلا باپ گرفتار', 'طالبہ اغوا کیس ؛ واردات میں ملوث مرکزی ملزم عابد سابقہ ریکارڈ یافتہ نکلا', 'امریکا میں پاکستانی نے بیوی ، بیٹی ساس کو قتل کرکے خودکشی کرلی', 'لاہور میں اغوا ہونے والی دسویں جماعت طالبہ بازیاب', 'رشتے بات چیت دوران تکرار ماں بیٹا قتل ، تین ملزمان گرفتار', 'پشاور میں گھر سے تھانے جاتے کار فائرنگ ، ایس ایچ او شہید', '11 سالہ بچے سے بدفعلی کرنے والا ہئیر ڈریسر گرفتار', 'کراچی میں سی ٹی ڈی آپریشن ؛ صدر دھماکے ماسٹر مائنڈ سمیت 2 دہشتگرد ہلاک', 'خاتون کو زیادتی بعد بلیک میل کرنے والا ساتھی ملزمہ سمیت گرفتار', 'قصبہ کالونی میں باپ نے غیرت نام بیٹی کو قتل کردیا', 'فیصل آباد میں 6 سالہ بچے سے مزدور مبینہ زیادتی'

In [9]:
df = pd.DataFrame({'Headline':myfiltered_sentence})
print(df.shape)
df.to_csv('newurdunews.csv', index=False)

(926, 1)


In [43]:
print(mydata['Headline'].tolist())

['ملازمہ کےمبینہ قتل ہولناک واردات', 'مناواں میں بہنوئی نے سالی کو قتل کردیا', 'منگنی توڑنے لڑکے نے منگیتر ساس کو قتل کر ڈالا', 'منگیتر بے وفائی کیخلاف لڑکی عدالت پہنچ', "میاں بیوی کےدرمیان صلح کرانا'پیر ' کو مہنگا پڑگیا", 'میجر ریٹائرڈ منیر احمد کو قتل کرنے والا ڈرائیور خاتون سمیت گرفتار', 'میو ہسپتال ڈاکٹر نعش واش روم سے برآمد', 'نامعلوم فائرنگ سے ریلوے انسپکٹر جاں بحق', 'نامعلوم شخص نے35 سالہ خاتون کا گلا کاٹ دیا', 'نصیر آباد میں تہرے قتل لرزہ خیز واردات', 'ننھی زینب سمیت دیگر بچیوں کا قاتل جنسی درندہ عمران کیفر کردار کو پہنچ گیا', 'ننھی عاصمہ کا قاتل ڈی این ٹیسٹ لودھراں سے لاہور منتقل', 'نوا ٹاؤن میں زمین کا تنازع زندگی نگل گیا', 'نواں کوٹ میں دوست نے دوست کو موت گھاٹ اتار دیا', 'ٹاؤن شپ میں دوہرے قتل لرزہ خیز واردات', 'ڈاکوؤں ہاتھوں پانچ بچوں ماں قتل', 'ڈولفن فورس نے شہری جان بچالی', 'ڈیفنس پرائیویٹ بینک میں صندوق سے سکیورٹی گارڈ لاش برآمد', 'ڈیفنس میں سالوں ہاتھوں بہنوئی قتل', 'کاہنہ میں دوہرے قتل لرزہ خیز واردات', 'کوٹ لکھپت جیل میں دیت رقم ادا کر بتیس قیدیوں کو رہا کردیا گیا'

In [98]:
X = data['Headline']
y = data['Category']
X.head()

0                     ملازمہ کےمبینہ قتل ہولناک واردات
1               مناواں میں بہنوئی نے سالی کو قتل کردیا
2        منگنی توڑنے لڑکے نے منگیتر ساس کو قتل کر ڈالا
3               منگیتر بے وفائی کیخلاف لڑکی عدالت پہنچ
4    میاں بیوی کےدرمیان صلح کرانا'پیر ' کو مہنگا پڑگیا
Name: Headline, dtype: object

In [99]:
# Split the data into 80-20 i.e. test size of 20% to check the accuracy of the training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

#Let's check the shape of the splitted data
print(f"Training Data Shape: {X_train.shape}")
print(f"Testing Data Shape: {X_test.shape}")

Training Data Shape: (824,)
Testing Data Shape: (206,)


In [100]:
# Let's first try with Count Vectorizer from scikit learn
cv = CountVectorizer()
X_train_cv = cv.fit_transform(X_train)
X_train_cv.shape

(824, 2029)

In [101]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_cv,y_train)

LinearSVC()

In [102]:
# Let's test it for the first 2 articles in the Test dataset
# X_test1 = X_test[0:5]
X_test1 = ['شہر مختلف مقامات سے 48 نامعلوم لاشیں برآمد', 'بے شرمی انتہا ، نجی سکول خواتین اساتذہ واش روم میں خفیہ کیمرے', 'مالکان کو لبھا کر فحش ویڈیوز بنانے والا لڑکیوں کا گروہ گرفتار', 'سنگین جرائم تفتیش کرنیوالے جدید سائنسی ادارے کیساتھ ہاتھ', 'ہوائی فائرنگ ویڈیو تصاویر وائرل کرنا نوجوان کو مہنگا پڑگیا', 'فوڈ اتھارٹی نے 800 کلو تیارجعلی شہد پکڑ لیا', 'لاہور پولیس کا ہائی پروفائل کریمینلز گرفتاری لیے پلان تیار', 'کوٹ لکھپت میں اغوا بعد ہمسائے 14 سالہ لڑکی سے زیادتی', 'تعویز بہانے بلاکر جعلی پیر خاتون سے زیادتی']
print(X_test1)

['شہر مختلف مقامات سے 48 نامعلوم لاشیں برآمد', 'بے شرمی انتہا ، نجی سکول خواتین اساتذہ واش روم میں خفیہ کیمرے', 'مالکان کو لبھا کر فحش ویڈیوز بنانے والا لڑکیوں کا گروہ گرفتار', 'سنگین جرائم تفتیش کرنیوالے جدید سائنسی ادارے کیساتھ ہاتھ', 'ہوائی فائرنگ ویڈیو تصاویر وائرل کرنا نوجوان کو مہنگا پڑگیا', 'فوڈ اتھارٹی نے 800 کلو تیارجعلی شہد پکڑ لیا', 'لاہور پولیس کا ہائی پروفائل کریمینلز گرفتاری لیے پلان تیار', 'کوٹ لکھپت میں اغوا بعد ہمسائے 14 سالہ لڑکی سے زیادتی', 'تعویز بہانے بلاکر جعلی پیر خاتون سے زیادتی']


In [104]:
# X_test1_cv = cv.transform(['شہر مختلف مقامات سے 48 نامعلوم لاشیں برآمد', 'بے شرمی انتہا ، نجی سکول خواتین اساتذہ واش روم میں خفیہ کیمرے', 'مالکان کو لبھا کر فحش ویڈیوز بنانے والا لڑکیوں کا گروہ گرفتار', 'سنگین جرائم تفتیش کرنیوالے جدید سائنسی ادارے کیساتھ ہاتھ', 'ہوائی فائرنگ ویڈیو تصاویر وائرل کرنا نوجوان کو مہنگا پڑگیا', 'فوڈ اتھارٹی نے 800 کلو تیارجعلی شہد پکڑ لیا', 'لاہور پولیس کا ہائی پروفائل کریمینلز گرفتاری لیے پلان تیار', 'کوٹ لکھپت میں اغوا بعد ہمسائے 14 سالہ لڑکی سے زیادتی', 'تعویز بہانے بلاکر جعلی پیر خاتون سے زیادتی'])
X_test1_cv = cv.transform(X_test1)
# clf.predict(X_test1_cv)
for i, x in enumerate(clf.predict(X_test1_cv)):
  print(i, x)

0 ظُلم و زیادتی
1 قتل وغارت
2 ظُلم و زیادتی
3 قتل وغارت
4 قتل وغارت
5 ظُلم و زیادتی
6 قتل وغارت
7 ظُلم و زیادتی
8 ظُلم و زیادتی


In [73]:
# Transform the test data before predicting
X_test_cv = cv.transform(X_test)

In [74]:
# Form a prediction set
predictions = clf.predict(X_test_cv)

In [75]:
# Report the confusion matrix
print(metrics.confusion_matrix(y_test,predictions))
# Print a classification report
print(metrics.classification_report(y_test,predictions))
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

[[32  1 13  0  2]
 [ 7  7  6  0  5]
 [10  0 72  1  5]
 [ 2  1  2  2  0]
 [ 1  3  5  0 29]]
               precision    recall  f1-score   support

ظُلم و زیادتی       0.62      0.67      0.64        48
    غیر متعلق       0.58      0.28      0.38        25
    قتل وغارت       0.73      0.82      0.77        88
نشہ آور اشیاء       0.67      0.29      0.40         7
         چوری       0.71      0.76      0.73        38

     accuracy                           0.69       206
    macro avg       0.66      0.56      0.59       206
 weighted avg       0.68      0.69      0.67       206

0.6893203883495146


In [22]:
# single command to create a pipeline of activities...vectorize and classify the text, in this case
clf_cvec_lsvc = Pipeline([('cvec', CountVectorizer()),
                     ('clf', LinearSVC())])

# Feed the training data through the pipeline
clf_cvec_lsvc.fit(X_train, y_train)

Pipeline(steps=[('cvec', CountVectorizer()), ('clf', LinearSVC())])

In [23]:
# Form a prediction set
# No need to convert the test data. Classifier cretaed in the pipeline will take care of it
predictions = clf_cvec_lsvc.predict(X_test)
# Report the confusion matrix
print(metrics.confusion_matrix(y_test,predictions))
# Print a classification report
print(metrics.classification_report(y_test,predictions))
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

[[32  1 13  0  2]
 [ 6  7  6  0  6]
 [10  0 72  1  5]
 [ 2  1  2  2  0]
 [ 1  3  5  0 29]]
               precision    recall  f1-score   support

ظُلم و زیادتی       0.63      0.67      0.65        48
    غیر متعلق       0.58      0.28      0.38        25
    قتل وغارت       0.73      0.82      0.77        88
نشہ آور اشیاء       0.67      0.29      0.40         7
         چوری       0.69      0.76      0.72        38

     accuracy                           0.69       206
    macro avg       0.66      0.56      0.58       206
 weighted avg       0.68      0.69      0.67       206

0.6893203883495146


In [102]:
clf_tfidf_lsvc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC())])

# Feed the training data through the pipeline
clf_tfidf_lsvc.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [103]:
# Form a prediction set
predictions = clf_tfidf_lsvc.predict(X_test)
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.6941747572815534


In [104]:
clf_tfidf_mnb = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', MultinomialNB())])

# Feed the training data through the pipeline
clf_tfidf_mnb.fit(X_train, y_train) 

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [105]:
# Form a prediction set
predictions = clf_tfidf_mnb.predict(X_test)
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.6699029126213593


In [106]:
clf_tfidf_lr = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LogisticRegression())])

# Feed the training data through the pipeline
clf_tfidf_lr.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LogisticRegression())])

In [107]:
predictions = clf_tfidf_lr.predict(X_test)
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.6990291262135923


In [108]:
clf_tfidf_knc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', KNeighborsClassifier())])

# Feed the training data through the pipeline
clf_tfidf_knc.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', KNeighborsClassifier())])

In [109]:
predictions = clf_tfidf_knc.predict(X_test)
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.616504854368932


In [110]:
clf_tfidf_rfc = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', RandomForestClassifier())])

# Feed the training data through the pipeline
clf_tfidf_rfc.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', RandomForestClassifier())])

In [111]:
predictions = clf_tfidf_rfc.predict(X_test)
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.6893203883495146


In [8]:
myfiltered_sentence = []
def cleaner(article):
    article = re.sub(r'\([^)]*\)', r'', article)
    article = re.sub(r'\[[^\]]*\]', r'', article)
    article = re.sub(r'<[^>]*>', r'', article)
    article = re.sub(r'^https?:\/\/.*[\r\n]*', '', article)
    article = article.replace(u'\ufeff','')
    article = article.replace(u'\xa0', u' ')
    article = article.replace('  ', ' ');
    article = article.replace(' , ', ', ');
    return article

for i, x in enumerate(filtered_sentence):
    myfiltered_sentence.append(cleaner(x))
print(myfiltered_sentence)

['ننکانہ صاحب میں بھائی نے 2 بہنوں کو قتل کردیا', 'سوتیلے بیٹے ہاتھوں ماں قتل میں سنسنی خیز انکشاف', 'لاہور میں پولیس اہلکار نے لین دین تنازع کراچی نوجوان کو قتل کردیا', 'لاہور میں پولیس کارروائی ، ڈکیت میاں بیوی گرفتار', 'پتوکی میں ڈاکوؤں 15 سالہ لڑکی سے زیادتی ، وزیراعلی لڑکی گھر پہنچ', 'راولپنڈی میں فائرنگ سے بچے قتل', 'کراچی میں زنجیروں سے بندھی بچی بازیاب ، ماں سوتیلا باپ گرفتار', 'طالبہ اغوا کیس ؛ واردات میں ملوث مرکزی ملزم عابد سابقہ ریکارڈ یافتہ نکلا', 'امریکا میں پاکستانی نے بیوی ، بیٹی ساس کو قتل کرکے خودکشی کرلی', 'لاہور میں اغوا ہونے والی دسویں جماعت طالبہ بازیاب', 'رشتے بات چیت دوران تکرار ماں بیٹا قتل ، تین ملزمان گرفتار', 'پشاور میں گھر سے تھانے جاتے کار فائرنگ ، ایس ایچ او شہید', '11 سالہ بچے سے بدفعلی کرنے والا ہئیر ڈریسر گرفتار', 'کراچی میں سی ٹی ڈی آپریشن ؛ صدر دھماکے ماسٹر مائنڈ سمیت 2 دہشتگرد ہلاک', 'خاتون کو زیادتی بعد بلیک میل کرنے والا ساتھی ملزمہ سمیت گرفتار', 'قصبہ کالونی میں باپ نے غیرت نام بیٹی کو قتل کردیا', 'فیصل آباد میں 6 سالہ بچے سے مزدور مبینہ زیادتی'

In [146]:
listToStr = ' '.join([str(elem) for elem in F])
print(listToStr)

ملازمہ کےمبینہ قتل کی ہولناک واردات مناواں میں بہنوئی نے سالی کو قتل کردیا منگنی توڑنے پر لڑکے نے منگیتر اور ساس کو قتل کر ڈالا منگیتر کی بے وفائی کیخلاف لڑکی عدالت پہنچ گئی میاں بیوی کےدرمیان صلح کرانا'پیر 'کو مہنگا پڑگیا میجر ریٹائرڈ منیر احمد کو قتل کرنے والا ڈرائیور خاتون سمیت گرفتار میو ہسپتال کے ڈاکٹر کی نعش واش روم سے برآمد نامعلوم افراد کی فائرنگ سے ریلوے انسپکٹر جاں بحق نامعلوم شخص نے35 سالہ خاتون کا گلا کاٹ دیا نصیر آباد میں تہرے قتل کی لرزہ خیز واردات ننھی زینب سمیت دیگر بچیوں کا قاتل جنسی درندہ عمران کیفر کردار کو پہنچ گیا ننھی عاصمہ کا قاتل ڈی این ٹیسٹ کیلئے لودھراں سے لاہور منتقل نوا ب ٹاؤن میں زمین کا تنازع زندگی نگل گیا نواں کوٹ میں دوست نے دوست کو موت کے گھاٹ اتار دیا ٹاؤن شپ میں دوہرے قتل کی لرزہ خیز واردات ڈاکوؤں کے ہاتھوں پانچ بچوں کی ماں قتل ڈولفن فورس نے شہری کی جان بچالی ڈیفنس پرائیویٹ بینک میں صندوق سے سکیورٹی گارڈ کی لاش برآمد ڈیفنس میں سالوں کے ہاتھوں بہنوئی قتل کاہنہ میں دوہرے قتل کی لرزہ خیز واردات کوٹ لکھپت جیل میں دیت کی رقم ادا کر کے بتیس قیدیوں کو رہا 

True

['▁ملازمہ',
 '▁کے',
 'م',
 'بین',
 'ہ',
 '▁قتل',
 '▁کی',
 '▁ہول',
 'ناک',
 '▁واردات',
 '▁مناواں',
 '▁میں',
 '▁بہنو',
 'ئی',
 '▁نے',
 '▁سال',
 'ی',
 '▁کو',
 '▁قتل',
 '▁کردیا',
 '▁منگ',
 'نی',
 '▁توڑ',
 'نے',
 '▁پر',
 '▁لڑک',
 'ے',
 '▁نے',
 '▁منگ',
 'یت',
 'ر',
 '▁اور',
 '▁سا',
 'س',
 '▁کو',
 '▁قتل',
 '▁کر',
 '▁ڈال',
 'ا',
 '▁منگ',
 'یت',
 'ر',
 '▁کی',
 '▁بے',
 '▁و',
 'فائی',
 '▁کی',
 'خلا',
 'ف',
 '▁لڑکی',
 '▁عدالت',
 '▁پہنچ',
 '▁گئی',
 '▁می',
 'اں',
 '▁بیو',
 'ی',
 '▁کے',
 'در',
 'می',
 'ان',
 '▁ص',
 'لح',
 '▁کر',
 'انا',
 "'",
 'پیر',
 '▁',
 "'",
 'کو',
 '▁مہنگا',
 '▁پڑگ',
 'یا',
 '▁می',
 'جر',
 '▁ریٹ',
 'ائر',
 'ڈ',
 '▁من',
 'یر',
 '▁',
 'احم',
 'د',
 '▁کو',
 '▁قتل',
 '▁کرنے',
 '▁والا',
 '▁ڈرائیور',
 '▁خا',
 'ت',
 'ون',
 '▁سمیت',
 '▁گرفتار',
 '▁می',
 'و',
 '▁ہ',
 'سپ',
 'ت',
 'ال',
 '▁کے',
 '▁ڈاکٹر',
 '▁کی',
 '▁ن',
 'ع',
 'ش',
 '▁و',
 'اش',
 '▁روم',
 '▁سے',
 '▁برآمد',
 '▁نام',
 'عل',
 'وم',
 '▁افراد',
 '▁کی',
 '▁فائرنگ',
 '▁سے',
 '▁',
 'ریلو',
 'ے',
 '▁ان',
 'سپ',
 'کٹر',
 '▁ج',
 'اں